In [1]:
#  close all the active sessions:
import requests
import json
import sys
import os


In [ ]:
import sqlalchemy
try:
    # rollback any existing transaction
    if session.is_active:
        session.rollback()
        print("rolling back")

    # close the session
    session.close()
except Exception as e:
    print(f"Error when rolling back or closing session: {e}")

# dispose the engine
engine = session.get_bind()
engine.dispose()

# set session to None
session = None

# check again if session is still open
try:
    session.connection()
    print("Session is still open after disposing the engine")
except sqlalchemy.exc.InvalidRequestError:
    print("Session is closed after disposing the engine")
except AttributeError:
    print("Session is None")

In [2]:
# !pip install plotly

In [3]:
#  roll back the session:

json_data=None

In [4]:
import sys

sys.path.append("G-Ops_MVP")

In [ ]:
import pandas as pd
import json
import plotly.express as px
import copy
import datetime
import os
from datetime import datetime, timedelta

In [6]:
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy import create_engine
from mvp_app.models import Base

from dotenv import load_dotenv

load_dotenv()

# # Database setup
# DB_NAME="SATS-TESTDB"
# DB_HOST="sat-auroradb-test-instance-1.cdfee77nz8sl.us-east-1.rds.amazonaws.com"
# DB_USER="postgres"
# DB_PASS="satauroradbtest"

DB_NAME="SATS-STG"
DB_HOST="satsstgdbcluster-instance-1.c1cv1hpzppoz.ap-southeast-1.rds.amazonaws.com"
DB_USER="postgres"
DB_PASS="satauroradbstg"

DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = create_engine(DATABASE_URL)

Base.metadata.create_all(engine)
conn = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
import os

# Set an environment variable
os.environ['DB_NAME'] = DB_NAME
os.environ['DB_HOST'] = DB_HOST
os.environ['DB_USER'] = DB_USER
os.environ['DB_PASS'] = DB_PASS
os.environ['ROOT_PATH'] = "G:/G-Ops_MVP"

In [9]:
def task_input_to_dict(task_input):
    return {
        'uldBags': task_input.uld_bags,
        'flightNo': task_input.flight_no,
        'inboundFlightNo': task_input.inbound_flight_no,
        'outboundFlightNo': task_input.outbound_flight_no,
        'arrDep': task_input.arr_dep,
        'terminal': task_input.terminal,
        'taskDuration': task_input.task_duration,
        'pickupLocation': task_input.pickup_location,
        'dropoffLocation': task_input.dropoff_location,
        'departureLocation': task_input.departure_location,
        'noOfBags': task_input.no_of_bags,
        'wideNarrow': task_input.wide_narrow,
        'availablePickupTime': task_input.available_pickup_time,
        'taskDeadline': task_input.task_deadline,
        'excelIndex': task_input.excel_index,
    }

def load_data(file_path, sheet_name=None):
    
    """
    Load data from a specified file path. Determines if the file is a CSV or Excel based on the extension.
    For Excel files, a sheet name can be specified.

    Args:
    file_path (str): The path to the data file.
    sheet_name (str, optional): The sheet name to read from, for Excel files.

    Returns:
    DataFrame: A Pandas DataFrame loaded from the file.

    Raises:
    ValueError: If the file extension is not supported.
    """
    # Normalize the file path to handle paths uniformly
    # connect the path to the current working directory:
    #file_path = os.path.join(os.getcwd(), file_path)
    # Normalize the file path to handle paths uniformly
    file_path = file_path.strip()
    # Check the file extension and load data accordingly
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        return pd.read_excel(file_path, sheet_name=sheet_name)
    else:
        raise ValueError("Unsupported file format. Please provide a '.csv' or '.xls(x)' file.")

def dataframe_to_json(dataframe):
    """
    Write description for this function!--DONE
    Converts a filtered DataFrame to a JSON string. It filters flights based on specific criteria,
    validates completeness of required fields, and formats the result as a JSON list sorted by
    pickup and deadline times.

    Args:
        dataframe (pd.DataFrame): The pandas DataFrame containing flight data. It should include columns like
        'ArrivalFlightno.', 'Pickup Location', and others relevant for flight operations.

    Returns:
        str: A JSON-formatted string of flight data, sorted and indented for readability.

    Raises:
        ValueError: If required columns are missing or contain null values.
        KeyError: If JSON construction misses required keys.
        TypeError: If there are format issues with sorting fields.

    """
    # Convert each row to a JSON object and store them in a list
    data_list = []
    # Required columns list
    required_columns = [
        "ArrivalFlightno.", "Pickup Location", "Dropoff Location", "wide/narrow",
        "pickup available time", "Drop-off deadline"
    ]
    # Check for missing columns and raise an exception if any are missing
    missing_columns = [col for col in required_columns if col not in dataframe.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
    # Filter only SQ (WB+A380) +TR (WB) flights
    df = dataframe[((dataframe['ArrivalFlightno.'].str.startswith('SQ'))
            & (dataframe['wide/narrow'] != 'NB')
            ) |
    ((dataframe['ArrivalFlightno.'].str.startswith('TR')) & (dataframe['wide/narrow']=='WB'))]

    df = df[["ArrivalFlightno.", "Pickup Location", "Dropoff Location", "wide/narrow",
        "pickup available time", "Drop-off deadline"]]

    # Check for any missing values in the dataframe
    if df.isnull().values.any():
        raise ValueError("Missing values found in the dataframe")

    # Define the required keys expected in each JSON object
    required_keys = [
    'uldBags', 'flightNo', 'inboundFlightNo', 'outboundFlightNo',
    'arrDep', 'terminal', 'taskDuration', 'pickupLocation',
    'dropoffLocation', 'departureLocation', 'noOfBags', 'wideNarrow',
    'availablePickupTime', 'taskDeadline', 'excelIndex'
    ]

    data_list = []  # Initialize the data list for JSON data
    try:
        for index, row in df.iterrows():
            # Build the JSON data ensuring all keys are present using dict.get() method
            json_data = {
                key: row.get(key, f'Missing key: {key}') for key in required_keys
            }

            # Override specific key values with custom logic or formatting if necessary
            json_data.update({
                'uldBags': ['dummy'],  # Default or example value for 'uld_bags'
                'flightNo': row['ArrivalFlightno.'],
                'inboundFlightNo': '',
                'outboundFlightNo': '',
                'arrDep': 'arr',
                'terminal': 'dummy',
                'taskDuration': 1,
                'pickupLocation': row.get("Pickup Location", "(default pickup)"),
                'dropoffLocation': str(row.get("Dropoff Location", "(default dropoff)")),
                'departureLocation': 'dummy',
                'noOfBags': 1,
                'wideNarrow': row.get("wide/narrow", "N/A"),
                'availablePickupTime': pd.to_datetime(row.get('pickup available time', "2023-01-01")).isoformat(),
                'taskDeadline': pd.to_datetime(row.get('Drop-off deadline', "2023-01-01")).isoformat(),
                'excelIndex': str(index + 2)
            })

            # Check for any missing key notifications set by dict comprehension
            if any('Missing key:' in str(value) for value in json_data.values()):
                missing_keys = [key for key, value in json_data.items() if 'Missing key:' in value]
                raise KeyError(f"Missing required keys in JSON data: {', '.join(missing_keys)}")

            data_list.append(json_data)
    except KeyError as ke:
        raise KeyError(f"Key error in data: missing {ke} in JSON data. Ensure all required keys are present.")

    try:
        # Sort the list by pickup time and task deadline
        data_list = sorted(data_list, key=lambda x: (x["availablePickupTime"], x["taskDeadline"]))
    except TypeError as e:
        raise ValueError("Error sorting data list. Please check the format of 'available_pickup_time' and 'task_deadline': " + str(e))



    # data_list = json.dumps(data_list, indent=2)


    return data_list

def visualize_plotly(data):
    # Flattening JSON with driver ID
    all_tasks = []
    for driver, tasks in data['driver_tasks'].items():
      for task in tasks:
            task['driver'] = driver
            all_tasks.append(task)

    # Convert the list of tasks into a DataFrame
    df = pd.DataFrame(all_tasks)
    # Convert schedule times to datetime
    df['schedule_pickup_time'] = pd.to_datetime(df['schedule_pickup_time'],format='%Y-%m-%dT%H:%M:%S')
    df['schedule_dropoff_time'] = pd.to_datetime(df['schedule_dropoff_time'],format='%Y-%m-%dT%H:%M:%S')
    df['available_pickup_time'] = pd.to_datetime(df['available_pickup_time'],format='%Y-%m-%dT%H:%M:%S')
    df['task_deadline'] = pd.to_datetime(df['task_deadline'],format='%Y-%m-%dT%H:%M:%S')
            # Create a new column for the hover text
    df['hover_text'] = 'Driver: ' + df['driver'] + '<br>Flight_no.: ' + str(df['Flight_no.']) + \
                   '<br>Pickup Location: ' + df['pickup_location'].astype(str)+\
                     '<br>Dropoff Location: '+df['dropoff_location'].astype(str)+\
                    '<br> available pickup time: ' + df['available_pickup_time'].astype(str)+\
                    '<br> task deadline: ' + df['task_deadline'].astype(str) +\
                    '<br> priority: ' + (df['priority']).astype(str)
    # Sort drivers by the numerical part of the 'driver' string
    df['driver_id'] = df['driver'].str.extract('(\d+)').astype(int)
    df.sort_values('driver_id', inplace=True)
    # Extract the ordered list of drivers after sorting
    sorted_drivers = df.sort_values('driver_id')['driver'].unique()

    # Create the figure using the sorted list of drivers for y-axis category order
    fig = px.timeline(
        df,
        x_start='schedule_pickup_time',
        x_end='schedule_dropoff_time',
        y='driver',
        # color='Flight_no.',
        labels={'driver': 'Driver', 'flight_no': 'Flight No'},
        title='Driver Schedule',
        category_orders={'driver': sorted_drivers.tolist()},# Set the custom order for y-axis
        hover_data=['hover_text']
    )

    # Update layout to fit all driver labels and sort them
    fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray': sorted_drivers.tolist()})


    # Show the figure
    return fig,df


def save_solution_json(best_tasks, best_best_sol, folder_name):
    # Get the current time and format it
    current_time = datetime.datetime.now()
    time_suffix = current_time.strftime("%H_%M")

    # Generate the file name with the current time
    file_name = f'output_{time_suffix}.json'
    datetime_to_isoformat(best_tasks)

    # Prepare the data structure for JSON
    data = {
        "best_tasks": best_tasks,
        "driver_tasks":{},
        "best_schedule_sequence_for_each_driver": best_best_sol}
    
      
    # Nested structure for driver tasks
    for driver_id, schedule in best_best_sol.items():
        driver_key = f"driver{driver_id}"
        data["driver_tasks"][driver_key] = [best_tasks[get_task_index_from_id(best_tasks, j)] for j in schedule] if schedule else []
    # Write the JSON data to the file
    with open(f'{folder_name}/{file_name}', 'w') as file:
        json.dump(data, file, indent=4, default=str)

    print(f"Data written to '{folder_name}/{file_name}'.")
    return data

def datetime_to_isoformat(tasks):
    for task in tasks:
        if isinstance (task['task_deadline'], str):
            task['task_deadline'] = pd.to_datetime(task['task_deadline'])
        if isinstance (task['available_pickup_time'], str):
            task['available_pickup_time'] = pd.to_datetime(task['available_pickup_time'])
        task['task_deadline'] = task['task_deadline'].isoformat()
        task['available_pickup_time'] = task['available_pickup_time'].isoformat()
        try:
            task['schedule_pickup_time'] = task['schedule_pickup_time'].isoformat()
        except:
            task['schedule_pickup_time'] = None
        try:
            task['schedule_dropoff_time'] = task['schedule_dropoff_time'].isoformat()
        except:
            task['schedule_dropoff_time'] = None
    return tasks


def get_task_index_from_id(tasks, task_id):
    '''
    :param tasks (list[Dict]): list of tasks
    :param task_id (str): task id/excel_index
    '''
    for i, task in enumerate(tasks):
        if task['excel_index'] == str(task_id):
            return i



In [10]:
# import datetime 
# query_date = datetime.datetime.now() - datetime.timedelta(days=11)
# start_time = query_date.replace(hour=10, minute=0, second=0, microsecond=0).isoformat()
# end_time = query_date.replace(hour=12, minute=0, second=0, microsecond=0).isoformat()
# query_date

In [8]:
# start_time = '2025-01-22T00:00:00'
# start_time = '2025-01-22T00:20:00'
# end_time = '2025-01-22T05:00:00'
start_time = '2025-01-18T01:20:00'
end_time = '2025-01-18T04:00:00'

In [9]:
# end_time = '2025-01-23T06:00:00'

In [10]:
from sqlalchemy.sql import text

def fetch_task_from_to(start_time, end_time):
    query = text("SELECT * FROM tasks WHERE \
                 available_pickup_time >= :start_time \
                 AND available_pickup_time <= :end_time")
    result = conn.execute(query, {'start_time': start_time, 'end_time': end_time})
    columns = result.keys()
    result_dict = [dict(zip(columns, row)) for row in result]
    return result_dict
try:
    
    db_res = fetch_task_from_to(start_time, end_time)
except Exception as e:
    print(f"Error fetching data from database: {e}")
    session.rollback()
    session.close()
    

In [ ]:
db_res[:2]

In [ ]:
print(len(db_res))
#  convert db_res to dataframe:
df = pd.DataFrame(db_res)
df.tail()


In [12]:
def fetch_flight_from_to(db_res):
    flight_ids = [db["flight_id"] for db in db_res]
    
    if not flight_ids:
        return {}
    
    query = text("SELECT flight_id, flight_number, on_block, terminal, sta FROM flights WHERE flight_id IN :flight_ids")
    result = conn.execute(query, {'flight_ids': tuple(flight_ids)})
    
    # Get column names
    columns = result.keys()
    
    # Combine column names with row data to create dictionaries
    result_dict = {row[0]: {'flight_number': row[1], 'AIBT': row[2], 'terminal':row[3], 'sta': row[4]} for row in result}
    return result_dict
    
db_res_with_flight = fetch_flight_from_to(db_res)

In [ ]:
# print db_res_with_flight dictionary first elements:
for i in list(db_res_with_flight.items())[:2]:
    print(i)

In [ ]:
len(db_res_with_flight)

In [15]:
res = []
for i in db_res:
    if i['pickup_location'] != 'ADHOCB':
        res.append(i)


In [ ]:
len(res)

In [ ]:
db_res = res
#  conver db_res to dataframe:  
df = pd.DataFrame(db_res)
df.tail()

In [ ]:
# db_res
db_res_with_flight

In [ ]:
#  make se of all fligh_number in db_res_with_flight:
# access all keys in db_res_with_flight:
flight_numbers = list(db_res_with_flight.keys())
flight_numbers

In [ ]:
#  make se of all fligh_number in db_res:
flight_numbers = [db['flight_id'] for db in db_res]
flight_numbers_set_B= set(flight_numbers)
flight_numbers_set_B

In [ ]:
#Format the generated tasks to the format required by the task assignment model:
def model_input_transformation(task, flights):
        return {
                'uld_bags': '[["dummy"]]',  # Assuming this field is not in the DB, so setting it as an empty list
                'Flight_no.': flights[task['flight_id']]["flight_number"] if task['flight_id'] in flights else '',
                'AIBT': flights[task['flight_id']]["AIBT"].isoformat() if task['flight_id'] in flights else '',
                'inbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'outbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'arr_dep': 'arr',  # Assuming this field is not in the DB, so setting it as arr
                'terminal': flights[task['flight_id']]["terminal"] if task['flight_id'] in flights else '',  # Assuming this field is not in the DB, so setting it as empty
                'task_duration': 1,  # Assuming this field is not in the DB, so setting it as 0
                'pickup_location': 'ADHOCB' if task['pickup_location'] == 'T0' else task['pickup_location'] if task['pickup_location'] else 'ADHOCB',
                'dropoff_location': 'ADHOCB' if task['dropoff_location'] == 'T0' else task['dropoff_location'] if task['dropoff_location'] else 'ADHOCB',
                'departure_location': '',  # Assuming this field is not in the DB, so setting it as empty
                'no_of_bags': 1,  # Assuming this field is not in the DB, so setting it as 0
                'wide_narrow': 'WB',  # Assuming this field is not in the DB, so setting it as empty
                'available_pickup_time': task['available_pickup_time'].isoformat(),
                'task_deadline': task['task_deadline'],
                'main_deadline':task['main_deadline'] if task['main_deadline'] is not None else None,
                # 'priority': "", #Removed the field for now
                'priority': int(task['priority']) if task['priority'] is not None else 1,
                'trip_order': int(task['trip_order']) if task['trip_order'] is not None else None, 
                'excel_index': str(task['task_id'])  # Using task_id as excelIndex
            }
# Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
# db_res = [task for task in db_res if task['flight_id'] is not None]
# Apply the mapping function to each task
data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
json_data = data
if len(json_data) != 0:
    print(db_res[0])
    print(f"Number of tasks = {len(json_data)}")
    print(f"Number of flights = {len({d['Flight_no.'] for d in json_data})}")

In [22]:
def sorted_priority(json_data):
    df = pd.DataFrame(json_data)
    df['available_pickup_time2'] = pd.to_datetime(df['available_pickup_time'])
    df_sorted = df.sort_values(by='available_pickup_time')
    df_sorted = df_sorted.reset_index(drop=True)
    df_sorted['priority'] = df_sorted.index + 1
    task_list = []
    for index, row in df_sorted.iterrows():
        task = {'uld_bags': row['uld_bags'] if row['uld_bags'] is not None else '', 
          'Flight_no.': row['Flight_no.'] if row['Flight_no.'] is not None else '',
          'AIBT': row['AIBT'] if row['AIBT'] is not None else '',
          'inbound_flight_no':row['inbound_flight_no'] if row['inbound_flight_no'] is not None else '',
          'outbound_flight_no':row['outbound_flight_no'] if row['outbound_flight_no'] is not None else '',
          'arr_dep': row['arr_dep'] if row['arr_dep'] is not None else '',
          'terminal':row['terminal'] if row['terminal'] is not None else '',
          'task_duration': row['task_duration'] if row['task_duration'] is not None else '',
          'pickup_location': row['pickup_location'] if row['pickup_location'] is not None else '',
        'dropoff_location': row['dropoff_location'] if row['dropoff_location'] is not None else '',
        'departure_location':row['departure_location'] if row['departure_location'] is not None else '',
        'no_of_bags': row['no_of_bags'] if row['no_of_bags'] is not None else '',
        'wide_narrow': row['wide_narrow'] if row['wide_narrow'] is not None else '',
        'available_pickup_time': row['available_pickup_time'],
        'task_deadline': row['task_deadline'].to_pydatetime(),
        'main_deadline': row['main_deadline'].to_pydatetime() if row['main_deadline'] is not None else '',
        'priority': row['priority'],
        'trip_order': row['trip_order'],
        'excel_index': row['excel_index']}
        task_list.append(task)
    return task_list

json_data = sorted_priority(json_data)

In [ ]:
json_data[:2]

In [ ]:
len(json_data)

In [25]:
#Import the necessary modules:
#G-Ops_MVP/mvp_app/modules/task_assignment/rule_initialization.py
# import sys
# sys.path.append("G-Ops_MVP")
from mvp_app.modules.task_assignment.rule_initialization import unified_initialize
from mvp_app.utils.TA_utils import save_txt_and_visual
# from mvp_app.modules.task_assignment.arrdep_model.data import select_longest_travel_time_for_combined_task 

In [26]:
# wrong
# from sqlalchemy import MetaData, Table, select

# def current_generate_drivers_from_db(query_date):
#         # Define driver table structure
#         metadata = MetaData()
#         drivers_table = Table('drivers', metadata, autoload_with=session.bind)
#         driver_shifts_table = Table('driver_shifts', metadata, autoload_with=session.bind)
        
#         #Get date from current time:
#         current_date = query_date.date()
#         # Fetch driver shifts from the database and have a filter to check whether the shift start time and endtime are in the current date:
#         # Fetch driver shifts from the database and have a filter to check whether the shift start time and endtime are in the current date:
#         driver_shifts_query = session.execute(select([driver_shifts_table.c])).fetchall()
#         # driver_shifts_query = session.execute(select([driver_shifts_table])).fetchall()
#         filtered_driver_shifts = [dict(row) for row in driver_shifts_query if datetime.datetime.fromisoformat(str(row['shift_starttime'])).date() == current_date\
#                                    or datetime.datetime.fromisoformat(str(row['shift_endtime'])).date() == current_date]
        
#         # Get all_drivers in the format of: {'driver_id': {shift_start_time: '', shift_end_time: ''}}
#         all_drivers = {str(driver['driver_id']): {'shift_starttime': str(driver['shift_starttime']), 'shift_endtime': str(driver['shift_endtime']), 'position': None} for driver in filtered_driver_shifts}
        
#         #sort all_drivers based on shift_starttime:
#         all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1]['shift_starttime']))
#         return all_drivers

# print(f"querying drivers on {query_date.date()}")
# all_drivers = current_generate_drivers_from_db(query_date)
# print(all_drivers)

In [27]:
# import datetime
# query_date=datetime.datetime(2025, 1, 22, 14, 0, 58, 417746)
# query_date=datetime.datetime(2025, 1, 18, 0, 0, 0, 0)
# print(start_time, end_time)

In [ ]:
# Convert strings to datetime and adjust for 10 hours
# import datetime as datetime
from sqlalchemy import MetaData, Table, and_, cast, Date
from datetime import datetime, timedelta
start_time_n = datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%S') - timedelta(hours=24)
end_time_n = datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%S') + timedelta(hours=24)
# start_time = start_time - timedelta(hours=9)
# end_time = end_time + timedelta(hours=9)
print(f'shift_start_time={start_time_n}, end_time={end_time_n}')
def current_generate_drivers_from_db(start_time_n, end_time_n, session=session):
    # Define driver table structure
    print(f"Fetching drivers from the database for {start_time_n} to {end_time_n}")
    metadata = MetaData()
    drivers_table = Table('drivers', metadata, autoload_with=session.bind)
    driver_shifts_table = Table('driver_shifts', metadata, autoload_with=session.bind)
    print(f"Driver table columns: {drivers_table.columns.keys()}"
          f"\nDriver shifts table columns: {driver_shifts_table.columns.keys()}")

    try:
        filtered_driver_shift = session.query(driver_shifts_table).filter(
            and_(
                driver_shifts_table.c.shift_starttime >= start_time_n,
                driver_shifts_table.c.shift_endtime <= end_time_n
            )
        ).all()
        print(f"Number of drivers fetched: {len(filtered_driver_shift)}")

        # Get all_drivers in the format of: {'driver_id': {shift_start_time: '', shift_end_time: ''}}
        all_drivers = {str(driver.driver_id): {'shift_starttime': str(driver.shift_starttime), 'shift_endtime': str(driver.shift_endtime), 'position': None} for driver in filtered_driver_shift}

        #sort all_drivers based on shift_starttime:
        all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1]['shift_starttime']))
        return all_drivers
    except Exception as e:
        session.rollback()
        print(f"Error fetching drivers: {e}")
        return {}

print(f"Querying drivers from {start_time_n} to {end_time_n} for tasks {start_time}, end_time{end_time}")
all_drivers = current_generate_drivers_from_db(start_time_n,end_time_n)
#  show all_drivers as dataframe:
all_drivers_df = pd.DataFrame(all_drivers).T
all_drivers_df

In [ ]:
# keep only those drivers whose shift_end_time is greater than start_time:
# from datetime import datetime
start_time_n=start_time
# assuming start_time is a string in ISO format, convert it to datetime
start_time_n = datetime.fromisoformat(start_time_n)

all_drivers = {k:v for k,v in all_drivers.items() if datetime.fromisoformat(v['shift_endtime']) > start_time_n}
all_drivers_df = pd.DataFrame(all_drivers).T
print(type(start_time_n),type(start_time))


In [ ]:
# #  convert all_driver_df to dictionary:
# all_drivers = all_drivers_df.to_dict(orient='index')
all_drivers_df

In [ ]:
#  correct
from sqlalchemy import MetaData, Table, and_, cast, Date

def current_generate_drivers_from_db(query_date, session=session):
        # Define driver table structure
        print(f"Fetching drivers from the database for {query_date.date()}")
        metadata = MetaData()
        drivers_table = Table('drivers', metadata, autoload_with=session.bind)
        driver_shifts_table = Table('driver_shifts', metadata, autoload_with=session.bind)
        print(f"Driver table columns: {drivers_table.columns.keys()}"
              f"\nDriver shifts table columns: {driver_shifts_table.columns.keys()}")
        #Get date from current time:
        current_date = query_date.date()
        print(f"Current date: {current_date}")
      
        try:
          filtered_driver_shift = session.query(driver_shifts_table).filter(
          and_(
            cast(driver_shifts_table.c.shift_starttime, Date) == current_date,
            cast(driver_shifts_table.c.shift_endtime, Date) == current_date
          )
            ).all()
          print(f"Number of drivers fetched: {len(filtered_driver_shift)}")
      
          
          # Get all_drivers in the format of: {'driver_id': {shift_start_time: '', shift_end_time: ''}}
          all_drivers = {str(driver.driver_id): {'shift_starttime': str(driver.shift_starttime), 'shift_endtime': str(driver.shift_endtime), 'position': None} for driver in filtered_driver_shift}
          
          #sort all_drivers based on shift_starttime:
          all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1]['shift_starttime']))
          return all_drivers
        except Exception as e:
          session.rollback()
          print(f"Error fetching drivers: {e}")
          return {}

print(f"querying drivers on {query_date.date()}")
all_drivers = current_generate_drivers_from_db(query_date)
print(all_drivers)


In [ ]:
#  from all_drivers dictionary show shift_starttime and shift_endtime
for driver_id, driver in all_drivers.items():
    print(f"Driver ID: {driver_id}, Shift start time: {driver['shift_starttime']}, Shift end time: {driver['shift_endtime']}")


In [ ]:
len(all_drivers)

In [37]:
# def current_generate_drivers_from_db(query_date, session=session):
#     # Define driver table structure
#     print(f"Fetching drivers from the database for {query_date.date()}")
#     metadata = MetaData()
#     drivers_table = Table('drivers', metadata, autoload_with=session.bind)
#     driver_shifts_table = Table('driver_shifts', metadata, autoload_with=session.bind)
#     print(f"Driver table columns: {drivers_table.columns.keys()}"
#           f"\nDriver shifts table columns: {driver_shifts_table.columns.keys()}")
#     #Get date from current time:
#     current_date = query_date.date()
#     print(f"Current date: {current_date}")

#     try:
#         filtered_driver_shift = session.query(driver_shifts_table).filter(
#             and_(
#                 cast(driver_shifts_table.c.shift_starttime, Date) == current_date,
#                 cast(driver_shifts_table.c.shift_endtime, Date) == current_date
#             )
#         ).all()
#         print(f"Number of drivers fetched: {len(filtered_driver_shift)}")
#         #  sort all_drivers based on shift_starttime:
#         all_drivers = {str(driver.driver_id): {'shift_starttime': str(driver.shift_starttime), 'shift_endtime': str(driver.shift_endtime), 'position': None} for driver in filtered_driver_shift}
#         #  sort all_drivers based on shift_starttime:
#         all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1]['shift_starttime']))
#         # all_drivers = {str(driver['driver_id']): {'shift_starttime': str(driver['shift_starttime']), 'shift_endtime': str(driver['shift_endtime']), 'position': None} for driver in filtered_driver_shift}
    #     all_drivers = filtered_driver_shift
    #     # Get all_drivers in the format of: {'driver_id': {shift_start_time: '', shift_end_time: ''}}
    #     # all_drivers = {str(driver.'driver_id']): {'shift_starttime': str(driver[1]), 'shift_endtime': str(driver[2]), 'position': None} for driver in filtered_driver_shift}
    
    #     #sort all_drivers based on shift_starttime:
    #     # all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1][2]))
    #     return all_drivers
    # except Exception as e:
    #     session.rollback()  # Rollback the session to the state before the last commit
    #     print(f"Error fetching drivers: {e}")
    #     return {}

In [ ]:
all_drivers_df.head()

In [ ]:
# convert all_drivers_df to dictionary:
# all_drivers = all_drivers_df.to_dict(orient='index')
all_drivers

In [38]:
def datetime_to_isoformat(tasks):
    for task in tasks:
        if isinstance (task['task_deadline'], str):
            task['task_deadline'] = pd.to_datetime(task['task_deadline'])
        if isinstance (task['available_pickup_time'], str):
            task['available_pickup_time'] = pd.to_datetime(task['available_pickup_time'])
        task['task_deadline'] = task['task_deadline'].isoformat()
        task['available_pickup_time'] = task['available_pickup_time'].isoformat()
        try:
            task['schedule_pickup_time'] = task['schedule_pickup_time'].isoformat()
        except:
            task['schedule_pickup_time'] = None
        try:
            task['schedule_dropoff_time'] = task['schedule_dropoff_time'].isoformat()
        except:
            task['schedule_dropoff_time'] = None
    return tasks
def save_solution_json(best_tasks, best_best_sol, folder_name):
    # Get the current time and format it
    current_time = datetime.now()
    time_suffix = current_time.strftime("%H_%M")

    # Generate the file name with the current time
    file_name = f'output_{time_suffix}.json'
    datetime_to_isoformat(best_tasks)

    # Prepare the data structure for JSON
    data = {
        "best_tasks": best_tasks,
        "driver_tasks":{},
        "best_schedule_sequence_for_each_driver": best_best_sol}
    
      
    # Nested structure for driver tasks
    for driver_id, schedule in best_best_sol.items():
        driver_key = f"driver{driver_id}"
        data["driver_tasks"][driver_key] = [best_tasks[get_task_index_from_id(best_tasks, j)] for j in schedule] if schedule else []
    # Write the JSON data to the file
    with open(f'{folder_name}/{file_name}', 'w') as file:
        json.dump(data, file, indent=4, default=str)

    print(f"Data written to '{folder_name}/{file_name}'.")
    return data
def final_txt_and_json(best_tasks,best_best_sol,folder):
    # Replace colon character in folder string
    folder = folder.replace(':', '_')
    # Get the current timestamp
    # timestamp = datetime.datetime.now().strftime("%H%M%S")
    timestamp = datetime.now().strftime("%H%M%S")
    # Create a folder with the timestamp as the name
    folder_name = f"{folder}-{timestamp}"
    os.makedirs(folder_name,exist_ok=True)
    # Save results to text file and visualize the schedule
    return save_solution_json(best_tasks, best_best_sol,folder_name)
def get_task_index_from_id(tasks, task_id):
    '''
    :param tasks (list[Dict]): list of tasks
    :param task_id (str): task id/excel_index
    '''
    for i, task in enumerate(tasks):
        if task['excel_index'] == str(task_id):
            return i
# call function to see the folder created:


In [ ]:
def test_model(data,reassignment, num_driver_range,start_time=None, end_time=None,lock_chromo={},adjustable_lock_tasks=[]):
    excel_data = pd.read_excel('travel_time.xlsx',usecols=['PickUp Location','DropOff Location','TravelTime'])  # Replace 'average_travel_time.xlsx' with your file path
    # convert all 'PickUp Location' and 'DropOff Location' into string
    excel_data['PickUp Location'] = excel_data['PickUp Location'].astype(str)
    excel_data['DropOff Location'] = excel_data['DropOff Location'].astype(str)
    excel_data = excel_data.drop_duplicates(subset=['PickUp Location','DropOff Location'], keep="last")
    excel_data['TravelTime'] = excel_data['TravelTime'].round(1)
    excel_data = excel_data.dropna(axis=0)
    # print(excel_data.head())
    #Pass in the drivers information here:
    #Tried for type=1 initialisation so far.
    baseline_model_result = [unified_initialize(copy.deepcopy(data),type=1,\
                                                current_time=pd.to_datetime(start_time), \
                                                end_time=pd.to_datetime(end_time),\
                                                num_driver_range=num_driver_range,\
                                                all_drivers=all_drivers, \
                                                rule_initialize_population_by_flight_parameter=False, \
                                                reassignment=reassignment,\
                                                lock_chromosome = lock_chromo,\
                                               adjustable_lock_tasks = adjustable_lock_tasks),]
                            
    folder_name='Model/model_result'
    file_name = f'new_taskAssignment_{start_time}_{end_time}'
    # Get the current date and time
    # current_date_time = datetime.datetime.now().replace(microsecond=0).isoformat()
    current_date_time = datetime.now().replace(microsecond=0).isoformat()
    # # # sort by number of driver requird and then penalty
    for i in baseline_model_result:
        print(i[0], i[3])
    baseline_model_result.sort(key = lambda x: (x[0] if x[0] is not None else float('inf'), x[3] if x[3] is not None else float('inf')))
    #save the model with (least number of driver, (schedule, penalty))
    result = final_txt_and_json(copy.deepcopy(baseline_model_result[0][2]),baseline_model_result[0][1], folder=f'{folder_name}/{file_name}')
    return result
try:
    lock_chromo = {}
    reassignment= False  
    adjustable_lock_tasks = []
    result = test_model(json_data,reassignment,num_driver_range=range(4,40), start_time = start_time,end_time = end_time, lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)
except KeyboardInterrupt:
    print("KeyboardInterrupt: Stopping the model run.")
    session.rollback()
    session.close()
except Exception as e :
    session.rollback() # This is where you roll back the session
    if session.is_active:
        session.rollback()
    print(f"Error : {e}")
finally:
    session.close()

In [ ]:

try:
    lock_chromo = {}
    reassignment= False  
    adjustable_lock_tasks = []
    result = test_model(json_data,reassignment,num_driver_range=range(4,40), start_time = start_time,end_time = end_time, lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)
except KeyboardInterrupt:
    print("KeyboardInterrupt: Stopping the model run.")
    session.rollback()
    session.close()
except Exception as e :
    session.rollback() # This is where you roll back the session
    if session.is_active:
        session.rollback()
    print(f"Error : {e}")
finally:
    session.close() # Ensure session is closed in the end
    

In [ ]:
# with open("Gerry/model_result/7drivers_21-23_14_aug/output_08_56.json", 'r') as file:
#     result = json.load(file)
fig, df = visualize_plotly(result)
fig.show()

In [15]:
import os

def save_solution_json(folder_name):
    # Get the current time and format it
    current_time = datetime.now()
    time_suffix = current_time.strftime("%Y%m%d_%H%M%S")

    # Generate the file name with the current time
    file_name = f'on_time_{time_suffix}.json'

    # Here you would typically write data to the file
    # For this example, we'll just print the file name
    print(f"Data would be written to '{folder_name}/{file_name}'.")

def create_folder_and_save_json(folder):
    # Replace problematic characters in folder string
    folder = folder.replace(':', '_').replace(' ', '_').replace('-', '_')

    # Create a folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Save results to json file
    save_solution_json(folder)

start_time = '2025-01-18T01:20:00'
end_time = '2025-01-18T04:00:00'
folder_name = 'Model/model_result'
file_name = f'{start_time}_to_{end_time}'
file_name = file_name.replace('T', '_')

create_folder_and_save_json(folder=f'{folder_name}/{file_name}')


Data would be written to 'Model/model_result/2025_01_18_01_20_00_to_2025_01_18_04_00_00/on_time_20250131_110549.json'.


In [ ]:
import pandas as pd
import os
from datetime import datetime

def create_folder_and_save_excel(data, folder, filename):
    # Replace problematic characters in folder string
    folder = folder.replace(':', '_').replace(' ', '_').replace('-', '_')

    # Create a folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert datetime objects to string
    df['task_deadline'] = df['task_deadline'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    df['main_deadline'] = df['main_deadline'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

    # Save the DataFrame as an Excel file in the specified folder
    try:
        df.to_excel(os.path.join(folder, filename), index=False)
        print("excel file saved with name: ", filename) 
    except exception as e:
        print(f"Error saving Excel file: {e}")
        

# Test the function


create_folder_and_save_excel(json_data, 'my_folder', 'my_file.xlsx')

In [ ]:
import math
def compute_deadline_breaches(tasks):
    breach_count = 0
    total_tasks = len(tasks)
    
    for task in tasks:
        if 'main_deadline' in task and task['main_deadline'] is not None:
            task_deadline = pd.to_datetime(task['main_deadline'], format='%Y-%m-%dT%H:%M:%S')
        else:
            task_deadline = pd.to_datetime(task['task_deadline'], format='%Y-%m-%dT%H:%M:%S')
        scheduled_dropoff = pd.to_datetime(task['schedule_dropoff_time'], format='%Y-%m-%dT%H:%M:%S')
        
        if scheduled_dropoff > task_deadline:
            # print(f"problematic task is {task}")
            breach_count += 1
    
    if total_tasks == 0:
        return 0  # Avoid division by zero
    
    breach_percentage = (breach_count / total_tasks) * 100
    return breach_count, breach_percentage

def compute_deadline_breach_time(tasks): 
    breache_time = 0 
    
    for task in tasks:
        if 'main_deadline' in task and task['main_deadline'] is not None:
            task_deadline = pd.to_datetime(task['main_deadline'], format='%Y-%m-%dT%H:%M:%S')
        else:
            task_deadline = pd.to_datetime(task['task_deadline'], format='%Y-%m-%dT%H:%M:%S')
        scheduled_dropoff = pd.to_datetime(task['schedule_dropoff_time'], format='%Y-%m-%dT%H:%M:%S')
        
        if scheduled_dropoff > task_deadline:
            breache_time += int((scheduled_dropoff - task_deadline).total_seconds()/60)
    
    return breache_time

def compute_drivers_breaches(driver_tasks):
    '''
    driver_tasks is a dictionary with driver id as a key and list of tasks
    as the value. Compute number of driver with zero deadline breaches and with deadline breaches
    '''
    zero_breach_count = 0
    driver_list = []
    for driver, tasks in driver_tasks.items():
        breach_count, breach_percentage = compute_deadline_breaches(tasks)
        if math.isclose(breach_percentage, 0., rel_tol=1e-5):
            zero_breach_count += 1
        
        driver_list.append(driver)
    
    breach_count = len(driver_list) - zero_breach_count
    
    return zero_breach_count, breach_count

def check_terminal_between_pairs_of_loc(pickup_location, dropoff_location):
    T2_location = ['Holding Area 2', 'ODD2']
    T3_location = ['Holding Area 3', 'ODD3']
    terminal_type = { 'A':'T3', 'B':'T3', 'E':'T2', "F":'T2', 'C':'T1','D':'T1'} 
    if pickup_location in T2_location:
        pickup_terminal = 'T2'
    elif pickup_location[0] in terminal_type:
        pickup_terminal = terminal_type[pickup_location[0]]
    elif pickup_location in T3_location:
        pickup_terminal = 'T3'
    elif pickup_location =='T1' or pickup_location =='T2' or pickup_location =='T3':
        pickup_terminal = pickup_location
    elif len(pickup_location) == 2:
        if (pickup_location.startswith('1')) or (pickup_location.startswith('2')):
            pickup_terminal = 'T1'
        elif pickup_location.startswith('3'):
            pickup_terminal = 'T2'
        elif pickup_location.startswith('4'):
            pickup_terminal = 'T3'
    else:
        print('Unregister pickup location. Taking T2 as default ',pickup_location)
        pickup_terminal = 'T2'

    if dropoff_location in T2_location:  
        dropoff_terminal = 'T2'
    elif dropoff_location[0] in terminal_type:
        dropoff_terminal = terminal_type[dropoff_location[0]]
    elif dropoff_location in T3_location:
        dropoff_terminal = 'T3'
    elif dropoff_location =='T1' or dropoff_location =='T2' or dropoff_location =='T3':
        dropoff_terminal = dropoff_location
    elif len(dropoff_location) == 2:
        if (dropoff_location.startswith('1')) or (dropoff_location.startswith('2')):
            dropoff_terminal = 'T1'
        elif dropoff_location.startswith('3'):
            dropoff_terminal = 'T2'
        elif dropoff_location.startswith('4'):
            dropoff_terminal = 'T3'
    else:
        print('Unregister dropoff location, Taking T2 as default ',dropoff_location)
        dropoff_terminal = 'T2'
    return pickup_terminal == dropoff_terminal

def compute_xt_between_tasks(driver_tasks):
    '''
    Only check between previous task dropoff loc and current task pickup loc
    '''
    total_tasks = 0
    num_xt = 0

    for driver_id, tasks in driver_tasks.items():
        tasks = sorted(tasks, key=lambda x: x['schedule_pickup_time'])  # Ensure tasks are sorted by pickup time
        for i in range(len(tasks) - 1):
            current_task = tasks[i]
            next_task = tasks[i + 1]

            dropoff_location = current_task['dropoff_location']
            pickup_location = next_task['pickup_location']
            if not check_terminal_between_pairs_of_loc(dropoff_location, pickup_location):
                num_xt += 1
            total_tasks += 1
            
    if total_tasks != 0:
        percentage = (num_xt / total_tasks) * 100
    else:
        percentage = 0
        
    return num_xt, percentage

def compute_drivers_xt(driver_tasks):
    xt_count = 0
    driver_list = []
    
    
    for driver, tasks in driver_tasks.items():
        total_tasks = 0
        num_xt = 0
        tasks = sorted(tasks, key=lambda x: x['schedule_pickup_time'])  # Ensure tasks are sorted by pickup time
        for i in range(len(tasks) - 1):
            current_task = tasks[i]
            next_task = tasks[i + 1]

            dropoff_location = current_task['dropoff_location']
            pickup_location = next_task['pickup_location']
            if not check_terminal_between_pairs_of_loc(dropoff_location, pickup_location):
                num_xt += 1
            total_tasks += 1
            
        if total_tasks != 0:
            xt_percentage=(num_xt / total_tasks) * 100
            # print(xt_percentage) 
            if not math.isclose(xt_percentage, 0., rel_tol=1e-5):
                xt_count += 1
                # print(xt_count)
        
        driver_list.append(driver)
    
    zero_xt = len(driver_list) - xt_count
    
    return xt_count, zero_xt


In [ ]:
tasks_list = result["best_tasks"]
driver_tasks = result["driver_tasks"]
breach_count, breach_percentage = compute_deadline_breaches(tasks_list)
zero_breach_count, drivers_breach_count = compute_drivers_breaches(driver_tasks)
num_xt, xt_percentage = compute_xt_between_tasks(driver_tasks)
xt_count, zero_xt  = compute_drivers_xt(driver_tasks)
print(f"Count of deadline breaches = {breach_count}")
print(f"Percentage of deadline breaches = {round(breach_percentage,2)} %")
print(f"Deadline breach time = {compute_deadline_breach_time(tasks_list)} minutes, {round(compute_deadline_breach_time(tasks_list)/60,2)} hours")
print(f"Number of drivers with zero deadline breach = {zero_breach_count}")
print(f"Number of drivers with deadline breach = {drivers_breach_count}")
print(f"Count of cross terminal = {num_xt}")
print(f"Percentage of xT trips/tasks = {round(xt_percentage,2)}%")
print(f"Number of drivers with zero cross terminal = {zero_xt}")
print(f"Number of drivers with cross terminal = {xt_count}")

In [ ]:
df = pd.DataFrame(result['best_tasks']) 
df.to_csv('new_Oct.9_22_24.csv', index=False)

In [ ]:
# scheduled_dropoff > task_deadline
breaches = 0
count = 0 
for tasks in result['best_tasks']:
    print(f"trip order: {tasks['trip_order']}")
    print(f"main deadline: {pd.to_datetime(tasks['main_deadline'])}")
    print(f"task deadline: {tasks['task_deadline']}")
    print(f"pick up: {tasks['schedule_pickup_time']}")
    print(f"drop off: {tasks['schedule_dropoff_time']}")
    time_delta = datetime.datetime.fromisoformat(tasks['schedule_dropoff_time']) - pd.to_datetime(tasks['main_deadline'])
    time_delta = time_delta.total_seconds()/60
    print('-----------------')
    if time_delta > 0:
        print(f"breaches: {time_delta} min")
        count += 1 
        breaches += time_delta 
    print('--------------')
    print(f'deadline breach task count:{count} ')
    print(f'deadline breach minutes: {breaches}')

In [ ]:
result['best_schedule_sequence_for_each_driver']

In [ ]:
# scheduled_dropoff > task_deadline
breaches = 0
count = 0 
for tasks in result['best_tasks']:
    print(f"trip order: {tasks['trip_order']}")
    print(f"main deadline: {pd.to_datetime(tasks['main_deadline'])}")
    print(f"task deadline: {tasks['task_deadline']}")
    print(f"pick up: {tasks['schedule_pickup_time']}")
    print(f"drop off: {tasks['schedule_dropoff_time']}")
    time_delta = datetime.datetime.fromisoformat(tasks['schedule_dropoff_time']) - pd.to_datetime(tasks['main_deadline'])
    time_delta = time_delta.total_seconds()/60
    print('-----------------')
    if time_delta > 0:
        print(f"breaches: {time_delta} min")
        count += 1 
        breaches += time_delta 
    print('--------------')
    print(f'deadline breach task count:{count} ')
    print(f'deadline breach minutes: {breaches}')

In [ ]:
# result['best_schedule_sequence_for_each_driver']

In [ ]:
# # scheduled_dropoff > task_deadline
# breaches = 0
# count = 0 
# for tasks in result['best_tasks']:
#     print(f"trip order: {tasks['trip_order']}")
#     print(f"main deadline: {pd.to_datetime(tasks['main_deadline'])}")
#     print(f"task deadline: {tasks['task_deadline']}")
#     print(f"pick up: {tasks['schedule_pickup_time']}")
#     print(f"drop off: {tasks['schedule_dropoff_time']}")
#     time_delta = datetime.datetime.fromisoformat(tasks['schedule_dropoff_time']) - pd.to_datetime(tasks['main_deadline'])
#     time_delta = time_delta.total_seconds()/60
#     print('-----------------')
#     if time_delta > 0:
#         print(f"breaches: {time_delta} min")
#         count += 1 
#         breaches += time_delta 
#     print('--------------')
#     print(f'deadline breach task count:{count} ')
#     print(f'deadline breach minutes: {breaches}')

In [ ]:
# from mvp_app.utils.TA_utils import find_traveltime

# terminal_type = { 'A':'T3', 'B':'T3', 'E':'T2', "F":'T2', 'C':'T1','D':'T1'}
# def calculate_preparation_time_breach(result_dict):
#     # Convert trip_time_df to a dictionary for quick lookup
#     # trip_time_dict = trip_time_df.set_index(['PickUp Location', 'DropOff Location'])['TravelTime'].to_dict()

#     total_tasks = 0
#     breaches = 0

#     for driver_id, tasks in result_dict.items():
#         tasks = sorted(tasks, key=lambda x: x['schedule_pickup_time'])  # Ensure tasks are sorted by pickup time
#         for i in range(len(tasks) - 1):
#             current_task = tasks[i]
#             next_task = tasks[i + 1]

#             dropoff_location = current_task['dropoff_location']
#             pickup_location = next_task['pickup_location']

#             # Get the trip time between the dropoff location and the pickup location
#             trip_time = find_traveltime(pickup_location=pickup_location, dropoff_location=dropoff_location)
#             if trip_time is None:
#                 print(f"I cannot find the trip time from {dropoff_location} to {pickup_location}")
#                 continue  # Skip if no trip time is found for the locations

#             # Calculate the time difference between the dropoff and the next pickup
#             time_diff = (pd.to_datetime(next_task['schedule_pickup_time'], format='%Y-%m-%dT%H:%M:%S')
#                          - pd.to_datetime(current_task['schedule_dropoff_time'], format='%Y-%m-%dT%H:%M:%S')).total_seconds() / 60  # Convert to minutes
#             time_diff = round(time_diff)
#             trip_time = round(trip_time)
#             if time_diff < trip_time and abs(time_diff-trip_time)>1e-3:
#                 breaches += 1

#             total_tasks += 1

#     # Calculate the percentage of breaches
#     if total_tasks == 0:
#         return 0.0
#     breach_percentage = (breaches / total_tasks) * 100
#     return breach_percentage

In [ ]:
# breach_percentage = calculate_preparation_time_breach(result["driver_tasks"])
# print(f'Preparation Time Breach Percentage: {breach_percentage:.2f}%') 

In [ ]:
# find_traveltime("T3", "T2") 

In [ ]:
# query_date = datetime.datetime.now() - datetime.timedelta(days=18)
# start_time = query_date.replace(hour=21, minute=0, second=0, microsecond=0).isoformat()
# end_time = query_date.replace(hour=23, minute=0, second=0, microsecond=0).isoformat()

# query_date 

## Simulation work 

In [ ]:
data = pd.read_csv('Tasks-data-1-Dec.csv') 
data['priority'] = 1 
data 

In [ ]:
df = data
df['available_pickup_time'] = pd.to_datetime(df['available_pickup_time'])

# 提取小时部分
df['hour'] = df['available_pickup_time'].dt.hour

# 定义划分时间段的函数
def assign_time_range(hour):
    return (hour // 2) * 2

# 添加时间范围列
df['time_range'] = df['hour'].apply(assign_time_range)

# 按照时间范围划分DataFrame
time_ranges = df['time_range'].unique()
dfs_by_time_range = {time_range: df[df['time_range'] == time_range] for time_range in time_ranges}

# 保存每个时间段的DataFrame为CSV文件
for time_range, df_range in dfs_by_time_range.items():
    file_name = f"time_range_{time_range}-{time_range + 2}.csv"
    df_range.to_csv(file_name, index=False)
    print(f"Saved {file_name}") 

In [ ]:
data = pd.read_csv('time_range_14-16.csv')   
data['priority'] = 1 
data 

In [ ]:
json_data = []
for index, row in data.iterrows():
    row_dict = row.to_dict()
    json_data.append(row_dict)

In [ ]:
from datetime import datetime 
def convert_to_datetime(time_str):
    try:
        return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')


time_fields = ['available_pickup_time', 'task_deadline', 'create_date', 'write_date', 'main_deadline']

In [ ]:
for row_dict in json_data:
    for field in time_fields:
        if isinstance(row_dict[field], str):
            row_dict[field] = convert_to_datetime(row_dict[field])

In [ ]:
def model_input_transformation(task):
        return {
                'uld_bags': '[["dummy"]]',  # Assuming this field is not in the DB, so setting it as an empty list
                'Flight_no.': task['flight_id'],
                'AIBT': '',
                'inbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'outbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'arr_dep': 'arr',  # Assuming this field is not in the DB, so setting it as arr
                'terminal': '',  # Assuming this field is not in the DB, so setting it as empty
                'task_duration': 1,  # Assuming this field is not in the DB, so setting it as 0
                'pickup_location': 'ADHOCB' if task['pickup_location'] == 'T0' else task['pickup_location'] if task['pickup_location'] else 'ADHOCB',
                'dropoff_location': 'ADHOCB' if task['dropoff_location'] == 'T0' else task['dropoff_location'] if task['dropoff_location'] else 'ADHOCB',
                'departure_location': '',  # Assuming this field is not in the DB, so setting it as empty
                'no_of_bags': 1,  # Assuming this field is not in the DB, so setting it as 0
                'wide_narrow': 'WB',  # Assuming this field is not in the DB, so setting it as empty
                'available_pickup_time': task['available_pickup_time'].isoformat(),
                'task_deadline': task['task_deadline'],
                'main_deadline':task['main_deadline'] if task['main_deadline'] is not None else None,
                # 'priority': "", #Removed the field for now
                'priority': int(task['priority']) if task['priority'] is not None else 1,
                'trip_order': int(task['trip_order']) if task['trip_order'] is not None else None, 
                'excel_index': str(task['task_id'])  # Using task_id as excelIndex
            }

In [ ]:
json_data = [model_input_transformation(task) for task in json_data] 

In [ ]:
def sorted_priority(json_data):
    df = pd.DataFrame(json_data)
    df['available_pickup_time2'] = pd.to_datetime(df['available_pickup_time'])
    df_sorted = df.sort_values(by='available_pickup_time')
    df_sorted = df_sorted.reset_index(drop=True)
    df_sorted['priority'] = df_sorted.index + 1
    task_list = []
    for index, row in df_sorted.iterrows():
        task = {'uld_bags': row['uld_bags'] if row['uld_bags'] is not None else '', 
          'Flight_no.': row['Flight_no.'] if row['Flight_no.'] is not None else '',
          'AIBT': row['AIBT'] if row['AIBT'] is not None else '',
          'inbound_flight_no':row['inbound_flight_no'] if row['inbound_flight_no'] is not None else '',
          'outbound_flight_no':row['outbound_flight_no'] if row['outbound_flight_no'] is not None else '',
          'arr_dep': row['arr_dep'] if row['arr_dep'] is not None else '',
          'terminal':row['terminal'] if row['terminal'] is not None else '',
          'task_duration': row['task_duration'] if row['task_duration'] is not None else '',
          'pickup_location': row['pickup_location'] if row['pickup_location'] is not None else '',
        'dropoff_location': row['dropoff_location'] if row['dropoff_location'] is not None else '',
        'departure_location':row['departure_location'] if row['departure_location'] is not None else '',
        'no_of_bags': row['no_of_bags'] if row['no_of_bags'] is not None else '',
        'wide_narrow': row['wide_narrow'] if row['wide_narrow'] is not None else '',
        'available_pickup_time': row['available_pickup_time'],
        'task_deadline': row['task_deadline'].to_pydatetime(),
        'main_deadline': row['main_deadline'].to_pydatetime() if row['main_deadline'] is not None else '',
        'priority': row['priority'],
        'trip_order': row['trip_order'],
        'excel_index': row['excel_index']}
        task_list.append(task)
    return task_list

json_data = sorted_priority(json_data) 

In [ ]:
json_data 

In [ ]:
len(json_data)

## 1. Test Deadline Breach and Cross Terminal with Sufficient Mock-up Drivers 

In [ ]:
NUM_DRIVER = 60
start_time_ = '1999-01-01T00:00:00'
end_time_ = '2100-01-01T00:00:00'
all_drivers = {}
for i in range(NUM_DRIVER):
    all_drivers[str(i)] = {'driver_id': str(i),
                          'shift_starttime':start_time_,
                          'shift_endtime': end_time_,
                          'position':None}

all_drivers

In [ ]:
from datetime import timedelta
import time
import csv 
TIME_DELTA = 1
query_date = datetime.datetime.now() - datetime.timedelta(days=TIME_DELTA)
csv_columns = [
    "query_date",
    "start_time", 
    "end_time",
    "Num_of_drivers",
    "Drivers",
    "Num_of_tasks",
    "Num_of_flights",
    "Count_of_deadline_breaches",
    "Percentage_of_deadline_breaches",
    "Deadline_breach_time_minutes",
    "Deadline_breach_time_hours",
    "Number_of_drivers_with_zero_deadline_breach",
    "Number_of_drivers_with_deadline_breach",
    "Count_of_cross_terminal",
    "Percentage_of_xT_trips_tasks",
    "Number_of_drivers_with_zero_cross_terminal",
    "Number_of_drivers_with_cross_terminal"
    ]

lock_chromo = {}
reassignment= False
adjustable_lock_tasks = []


for hour in range(0, 23):
    # get start time and end time for each 2-hour window
    if hour%2 ==0:
        start_time = query_date.replace(hour=hour, minute=0, second=0, microsecond=0).isoformat()
        if hour+2!=24:
            end_time = query_date.replace(hour=hour+2, minute=0, second=0, microsecond=0).isoformat()
        else:
            end_time = query_date.replace(hour=hour+1, minute=59, second=59, microsecond=0).isoformat()
        print(start_time)
        print(end_time)
        print('NEW SHIFT########################################################################')
    else: 
        continue  
    # # get tasks 
    # db_res = fetch_task_from_to(start_time, end_time)
    # # get flights
    # db_res_with_flight = fetch_flight_from_to(db_res)
    # # get formatted tsaks
    # #Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
    # # db_res = [task for task in db_res if task['flight_id'] is not None]
    # # Apply the mapping function to each task
    # data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
    # json_data = data
    # json_data = sorted_priority(json_data)
    # # print(json_data)
    
    if len(json_data) != 0:
        # print(db_res[0])
        task_num = len(json_data)
        flight_num = len({d['Flight_no.'] for d in json_data})
        print(f"Number of tasks = {task_num}")
        print(f"Number of flights = {flight_num}")
        # get drivers 
        # all_drivers = current_generate_drivers_from_db(query_date)
        print(f"Number of drivers = {len(all_drivers)}")

        
        # test model
        for driver_num in range(5, len(all_drivers)):
            # result = test_model(json_data, num_driver_range=range(driver_num, driver_num+1), start_time=start_time, end_time=end_time)
            result = test_model(json_data,reassignment,num_driver_range=range(driver_num, driver_num+1), start_time = start_time,end_time = end_time, lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)
            tasks_list = result["best_tasks"]
            driver_tasks = result["driver_tasks"]
            drivers = result['best_schedule_sequence_for_each_driver'].keys()
            breach_count, breach_percentage = compute_deadline_breaches(tasks_list)
            zero_breach_count, drivers_breach_count = compute_drivers_breaches(driver_tasks)
            num_xt, xt_percentage = compute_xt_between_tasks(driver_tasks)
            xt_count, zero_xt  = compute_drivers_xt(driver_tasks)
            deadline_breach_time_minutes = compute_deadline_breach_time(tasks_list)
            deadline_breach_time_hours = round(deadline_breach_time_minutes / 60, 2)
            print(f"Count of deadline breaches = {breach_count}")
            print(f"Percentage of deadline breaches = {round(breach_percentage,2)} %")
            print(f"Deadline breach time = {deadline_breach_time_minutes} minutes, {round(deadline_breach_time_hours/60,2)} hours")
            print(f"Number of drivers with zero deadline breach = {zero_breach_count}")
            print(f"Number of drivers with deadline breach = {drivers_breach_count}")
            print(f"Count of cross terminal = {num_xt}")
            print(f"Percentage of xT trips/tasks = {round(xt_percentage,2)}%")
            print(f"Number of drivers with zero cross terminal = {zero_xt}")
            print(f"Number of drivers with cross terminal = {xt_count}")
            print(f'Drivers={drivers}')
            print('#########################################################################')

            data = {
                "query_date": query_date,          
                "start_time": start_time, 
                "end_time": end_time,
                "Num_of_drivers":driver_num,
                "Drivers":drivers,
                "Num_of_tasks":  task_num, 
                "Num_of_flights": flight_num,
                "Count_of_deadline_breaches": breach_count,
                "Percentage_of_deadline_breaches": round(breach_percentage, 2),
                "Deadline_breach_time_minutes": deadline_breach_time_minutes,
                "Deadline_breach_time_hours": deadline_breach_time_hours,
                "Number_of_drivers_with_zero_deadline_breach": zero_breach_count,
                "Number_of_drivers_with_deadline_breach": drivers_breach_count,
                "Count_of_cross_terminal": num_xt,
                "Percentage_of_xT_trips_tasks": round(xt_percentage, 2),
                "Number_of_drivers_with_zero_cross_terminal": zero_xt,
                "Number_of_drivers_with_cross_terminal": xt_count}

            with open(f'tasks_report_{query_date}.csv', 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                if csvfile.tell() == 0:
                    writer.writeheader()
                writer.writerow(data)

    else:
        print('There is no task')

In [ ]:
TIME_DELTA = 12
query_date = datetime.datetime.now() - datetime.timedelta(days=TIME_DELTA)
query_date

In [ ]:
from datetime import timedelta
import time
import csv 
TIME_DELTA = 1
query_date = datetime.datetime.now() - datetime.timedelta(days=TIME_DELTA)
csv_columns = [
    "query_date",
    "start_time", 
    "end_time",
    "Num_of_drivers",
    "Drivers",
    "Num_of_tasks",
    "Num_of_flights",
    "Count_of_deadline_breaches",
    "Percentage_of_deadline_breaches",
    "Deadline_breach_time_minutes",
    "Deadline_breach_time_hours",
    "Number_of_drivers_with_zero_deadline_breach",
    "Number_of_drivers_with_deadline_breach",
    "Count_of_cross_terminal",
    "Percentage_of_xT_trips_tasks",
    "Number_of_drivers_with_zero_cross_terminal",
    "Number_of_drivers_with_cross_terminal"
    ]

lock_chromo = {}
reassignment= False
adjustable_lock_tasks = []


for hour in range(14, 23):
    # get start time and end time for each 2-hour window
    if hour%2 ==0:
        start_time = query_date.replace(hour=hour, minute=0, second=0, microsecond=0).isoformat()
        if hour+2!=24:
            end_time = query_date.replace(hour=hour+2, minute=0, second=0, microsecond=0).isoformat()
        else:
            end_time = query_date.replace(hour=hour+1, minute=59, second=59, microsecond=0).isoformat()
        print(start_time)
        print(end_time)
        print('NEW SHIFT########################################################################')
    else: 
        continue  
    # get tasks 
    db_res = fetch_task_from_to(start_time, end_time)
    # get flights
    db_res_with_flight = fetch_flight_from_to(db_res)
    # get formatted tsaks
    #Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
    # db_res = [task for task in db_res if task['flight_id'] is not None]
    # Apply the mapping function to each task
    data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
    json_data = data
    json_data = sorted_priority(json_data)
    # print(json_data)
    
    if len(json_data) != 0:
        # print(db_res[0])
        task_num = len(json_data)
        flight_num = len({d['Flight_no.'] for d in json_data})
        print(f"Number of tasks = {task_num}")
        print(f"Number of flights = {flight_num}")
        # get drivers 
        # all_drivers = current_generate_drivers_from_db(query_date)
        print(f"Number of drivers = {len(all_drivers)}")

        
        # test model
        for driver_num in range(2, 30):
            # result = test_model(json_data, num_driver_range=range(driver_num, driver_num+1), start_time=start_time, end_time=end_time)
            result = test_model(json_data,reassignment,num_driver_range=range(driver_num, driver_num+1), start_time = start_time,end_time = end_time, lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)
            tasks_list = result["best_tasks"]
            driver_tasks = result["driver_tasks"]
            drivers = result['best_schedule_sequence_for_each_driver'].keys()
            breach_count, breach_percentage = compute_deadline_breaches(tasks_list)
            zero_breach_count, drivers_breach_count = compute_drivers_breaches(driver_tasks)
            num_xt, xt_percentage = compute_xt_between_tasks(driver_tasks)
            xt_count, zero_xt  = compute_drivers_xt(driver_tasks)
            deadline_breach_time_minutes = compute_deadline_breach_time(tasks_list)
            deadline_breach_time_hours = round(deadline_breach_time_minutes / 60, 2)
            print(f"Count of deadline breaches = {breach_count}")
            print(f"Percentage of deadline breaches = {round(breach_percentage,2)} %")
            print(f"Deadline breach time = {deadline_breach_time_minutes} minutes, {round(deadline_breach_time_hours/60,2)} hours")
            print(f"Number of drivers with zero deadline breach = {zero_breach_count}")
            print(f"Number of drivers with deadline breach = {drivers_breach_count}")
            print(f"Count of cross terminal = {num_xt}")
            print(f"Percentage of xT trips/tasks = {round(xt_percentage,2)}%")
            print(f"Number of drivers with zero cross terminal = {zero_xt}")
            print(f"Number of drivers with cross terminal = {xt_count}")
            print(f'Drivers={drivers}')
            print('#########################################################################')

            data = {
                "query_date": query_date,          
                "start_time": start_time, 
                "end_time": end_time,
                "Num_of_drivers":driver_num,
                "Drivers":drivers,
                "Num_of_tasks":  task_num, 
                "Num_of_flights": flight_num,
                "Count_of_deadline_breaches": breach_count,
                "Percentage_of_deadline_breaches": round(breach_percentage, 2),
                "Deadline_breach_time_minutes": deadline_breach_time_minutes,
                "Deadline_breach_time_hours": deadline_breach_time_hours,
                "Number_of_drivers_with_zero_deadline_breach": zero_breach_count,
                "Number_of_drivers_with_deadline_breach": drivers_breach_count,
                "Count_of_cross_terminal": num_xt,
                "Percentage_of_xT_trips_tasks": round(xt_percentage, 2),
                "Number_of_drivers_with_zero_cross_terminal": zero_xt,
                "Number_of_drivers_with_cross_terminal": xt_count}

            with open(f'tasks_report_{query_date}.csv', 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                if csvfile.tell() == 0:
                    writer.writeheader()
                writer.writerow(data)

    else:
        print('There is no task')



In [ ]:
for i in task_list3:  
    db_res.append(i)

## 2. Test Deadline Breach and Cross Terminal with Real Drivers 

In [ ]:
# from datetime import timedelta
# import time
# import csv 
# import logging

# # Configure logging
# logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# TIME_DELTA = 6
# query_date = datetime.datetime.now() - datetime.timedelta(days=TIME_DELTA)
# csv_columns = [
#     "query_date",
#     "start_time", 
#     "end_time",
#     "Num_of_drivers",
#     "Num_of_tasks",
#     "Num_of_flights",
#     "Count_of_deadline_breaches",
#     "Percentage_of_deadline_breaches",
#     "Deadline_breach_time_minutes",
#     "Deadline_breach_time_hours",
#     "Number_of_drivers_with_zero_deadline_breach",
#     "Number_of_drivers_with_deadline_breach",
#     "Count_of_cross_terminal",
#     "Percentage_of_xT_trips_tasks",
#     "Number_of_drivers_with_zero_cross_terminal",
#     "Number_of_drivers_with_cross_terminal"
#     ]

# for hour in range(18, 23):
#     # get start time and end time for each 2-hour window
#     if hour%2 ==0:
#         start_time = query_date.replace(hour=hour, minute=0, second=0, microsecond=0).isoformat()
#         if hour+2!=24:
#             end_time = query_date.replace(hour=hour+2, minute=0, second=0, microsecond=0).isoformat()
#         else:
#             end_time = query_date.replace(hour=hour+1, minute=59, second=59, microsecond=0).isoformat()
#         print(start_time)
#         print(end_time)
#         print('NEW SHIFT$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
#     else: 
#         continue  
#     # get tasks 
#     db_res = fetch_task_from_to(start_time, end_time)
#     # get flights
#     db_res_with_flight = fetch_flight_from_to(db_res)
#     # get formatted tsaks
#     #Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
#     # db_res = [task for task in db_res if task['flight_id'] is not None]
#     # Apply the mapping function to each task
#     data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
#     json_data = data
#     # print(json_data)
    
#     if len(json_data) != 0:
#         # print(db_res[0])
#         task_num = len(json_data)
#         flight_num = len({d['Flight_no.'] for d in json_data})
#         print(f"Number of tasks = {task_num}")
#         print(f"Number of flights = {flight_num}")
#         # get drivers 
#         all_drivers = current_generate_drivers_from_db(query_date)
#         print(f"Number of drivers = {len(all_drivers)}")

        
#         # test model
#         for driver_num in range(1, len(all_drivers)+1):
#             try: 
#                 result = test_model(json_data, num_driver_range=range(driver_num, driver_num+1), query_date=query_date, start_time=start_time, end_time=end_time)
#                 tasks_list = result["best_tasks"]
#                 driver_tasks = result["driver_tasks"]
#                 breach_count, breach_percentage = compute_deadline_breaches(tasks_list)
#                 zero_breach_count, drivers_breach_count = compute_drivers_breaches(driver_tasks)
#                 num_xt, xt_percentage = compute_xt_between_tasks(driver_tasks)
#                 xt_count, zero_xt  = compute_drivers_xt(driver_tasks)
#                 deadline_breach_time_minutes = compute_deadline_breach_time(tasks_list)
#                 deadline_breach_time_hours = round(deadline_breach_time_minutes / 60, 2)
#                 print(f"Count of deadline breaches = {breach_count}")
#                 print(f"Percentage of deadline breaches = {round(breach_percentage,2)} %")
#                 print(f"Deadline breach time = {deadline_breach_time_minutes} minutes, {round(deadline_breach_time_hours/60,2)} hours")
#                 print(f"Number of drivers with zero deadline breach = {zero_breach_count}")
#                 print(f"Number of drivers with deadline breach = {drivers_breach_count}")
#                 print(f"Count of cross terminal = {num_xt}")
#                 print(f"Percentage of xT trips/tasks = {round(xt_percentage,2)}%")
#                 print(f"Number of drivers with zero cross terminal = {zero_xt}")
#                 print(f"Number of drivers with cross terminal = {xt_count}")
#                 print('#########################################################################')

#                 data = {
#                     "query_date": query_date,          
#                     "start_time": start_time, 
#                     "end_time": end_time,
#                     "Num_of_drivers":driver_num,
#                     "Num_of_tasks":  task_num, 
#                     "Num_of_flights": flight_num,
#                     "Count_of_deadline_breaches": breach_count,
#                     "Percentage_of_deadline_breaches": round(breach_percentage, 2),
#                     "Deadline_breach_time_minutes": deadline_breach_time_minutes,
#                     "Deadline_breach_time_hours": deadline_breach_time_hours,
#                     "Number_of_drivers_with_zero_deadline_breach": zero_breach_count,
#                     "Number_of_drivers_with_deadline_breach": drivers_breach_count,
#                     "Count_of_cross_terminal": num_xt,
#                     "Percentage_of_xT_trips_tasks": round(xt_percentage, 2),
#                     "Number_of_drivers_with_zero_cross_terminal": zero_xt,
#                     "Number_of_drivers_with_cross_terminal": xt_count}

#                 with open(f'tasks_report_real_driver_{query_date}.csv', 'a', newline='') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     if csvfile.tell() == 0:
#                         writer.writeheader()
#                     writer.writerow(data)
                
#                 if (breach_percentage == 0) & (xt_percentage == 0):
#                     break 
                    
#             except Exception as e:
#                 logging.error(f"Error occurred with query_date={query_date} and from {start_time} to {end_time}: {e}")
#                 continue

#     else:
#         print('There is no task')


In [ ]:
# from datetime import timedelta
# import time
# import csv 
# import logging

# # Configure logging
# logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# TIME_DELTA = 18
# query_date = datetime.datetime.now() - datetime.timedelta(days=TIME_DELTA)
# csv_columns = [
#     "query_date",
#     "start_time", 
#     "end_time",
#     "Num_of_drivers",
#     "Num_of_tasks",
#     "Num_of_flights",
#     "Count_of_deadline_breaches",
#     "Percentage_of_deadline_breaches",
#     "Deadline_breach_time_minutes",
#     "Deadline_breach_time_hours",
#     "Number_of_drivers_with_zero_deadline_breach",
#     "Number_of_drivers_with_deadline_breach",
#     "Count_of_cross_terminal",
#     "Percentage_of_xT_trips_tasks",
#     "Number_of_drivers_with_zero_cross_terminal",
#     "Number_of_drivers_with_cross_terminal"
#     ]

# for hour in range(22, 23):
#     # get start time and end time for each 2-hour window
#     if hour%2 ==0:
#         start_time = query_date.replace(hour=hour, minute=0, second=0, microsecond=0).isoformat()
#         if hour+2!=24:
#             end_time = query_date.replace(hour=hour+2, minute=0, second=0, microsecond=0).isoformat()
#         else:
#             end_time = query_date.replace(hour=hour+1, minute=59, second=59, microsecond=0).isoformat()
#         print(start_time)
#         print(end_time)
#         print('NEW SHIFT$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
#     else: 
#         continue  
#     # get tasks 
#     db_res = fetch_task_from_to(start_time, end_time)
#     # get flights
#     db_res_with_flight = fetch_flight_from_to(db_res)
#     # get formatted tsaks
#     #Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
#     # db_res = [task for task in db_res if task['flight_id'] is not None]
#     # Apply the mapping function to each task
#     data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
#     json_data = data
#     # print(json_data)
    
#     if len(json_data) != 0:
#         # print(db_res[0])
#         task_num = len(json_data)
#         flight_num = len({d['Flight_no.'] for d in json_data})
#         print(f"Number of tasks = {task_num}")
#         print(f"Number of flights = {flight_num}")
#         # get drivers 
#         all_drivers = current_generate_drivers_from_db(query_date)
#         print(f"Number of drivers = {len(all_drivers)}")

        
#         # test model
#         for driver_num in range(1, len(all_drivers)+1):
#             try: 
#                 result = test_model(json_data, num_driver_range=range(driver_num, driver_num+1), query_date=query_date, start_time=start_time, end_time=end_time)
#                 tasks_list = result["best_tasks"]
#                 driver_tasks = result["driver_tasks"]
#                 breach_count, breach_percentage = compute_deadline_breaches(tasks_list)
#                 zero_breach_count, drivers_breach_count = compute_drivers_breaches(driver_tasks)
#                 num_xt, xt_percentage = compute_xt_between_tasks(driver_tasks)
#                 xt_count, zero_xt  = compute_drivers_xt(driver_tasks)
#                 deadline_breach_time_minutes = compute_deadline_breach_time(tasks_list)
#                 deadline_breach_time_hours = round(deadline_breach_time_minutes / 60, 2)
#                 print(f"Count of deadline breaches = {breach_count}")
#                 print(f"Percentage of deadline breaches = {round(breach_percentage,2)} %")
#                 print(f"Deadline breach time = {deadline_breach_time_minutes} minutes, {round(deadline_breach_time_hours/60,2)} hours")
#                 print(f"Number of drivers with zero deadline breach = {zero_breach_count}")
#                 print(f"Number of drivers with deadline breach = {drivers_breach_count}")
#                 print(f"Count of cross terminal = {num_xt}")
#                 print(f"Percentage of xT trips/tasks = {round(xt_percentage,2)}%")
#                 print(f"Number of drivers with zero cross terminal = {zero_xt}")
#                 print(f"Number of drivers with cross terminal = {xt_count}")
#                 print('#########################################################################')

#                 data = {
#                     "query_date": query_date,          
#                     "start_time": start_time, 
#                     "end_time": end_time,
#                     "Num_of_drivers":driver_num,
#                     "Num_of_tasks":  task_num, 
#                     "Num_of_flights": flight_num,
#                     "Count_of_deadline_breaches": breach_count,
#                     "Percentage_of_deadline_breaches": round(breach_percentage, 2),
#                     "Deadline_breach_time_minutes": deadline_breach_time_minutes,
#                     "Deadline_breach_time_hours": deadline_breach_time_hours,
#                     "Number_of_drivers_with_zero_deadline_breach": zero_breach_count,
#                     "Number_of_drivers_with_deadline_breach": drivers_breach_count,
#                     "Count_of_cross_terminal": num_xt,
#                     "Percentage_of_xT_trips_tasks": round(xt_percentage, 2),
#                     "Number_of_drivers_with_zero_cross_terminal": zero_xt,
#                     "Number_of_drivers_with_cross_terminal": xt_count}

#                 with open(f'tasks_report_real_driver_{query_date}.csv', 'a', newline='') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     if csvfile.tell() == 0:
#                         writer.writeheader()
#                     writer.writerow(data)
                
#                 if (breach_percentage == 0) & (xt_percentage == 0):
#                     break 
                    
#             except Exception as e:
#                 logging.error(f"Error occurred with query_date={query_date} and from {start_time} to {end_time}: {e}")
#                 continue

#     else:
#         print('There is no task')


# 4-02-2025


In [20]:
pwd

'g:\\'

In [1]:
import model_call

1
DB_NAME: SATS-STG
DB_HOST: satsstgdbcluster-instance-1.c1cv1hpzppoz.ap-southeast-1.rds.amazonaws.com
DB_USER: postgres
DB_PASS: satauroradbstg
ROOT_PATH: G:/G-Ops_MVP


In [2]:
# import sys
# sys.path.append("G-Ops_MVP")
import json
# import pytest
import pandas as pd
from datetime import datetime


In [3]:
import pandas as pd
from datetime import datetime

def load_data(file_path):
    tasks_df = pd.read_excel(file_path, sheet_name="Generated tasks")
    drivers_df = pd.read_excel(file_path, sheet_name="Drivers")

    tasks_data = []
    for _, row in tasks_df.iterrows():
        try:  # Add a try-except block for date parsing
            available_pickup_time = row['available_pickup_time']
            task_deadline = row['task_deadline']
            main_deadline = row['main_deadline']

            # Convert to datetime objects, handling potential errors
            # available_pickup_time = pd.to_datetime(available_pickup_time) if not isinstance(available_pickup_time, datetime) else available_pickup_time
            task_deadline = pd.to_datetime(task_deadline) if not isinstance(task_deadline, datetime) else task_deadline
            main_deadline = pd.to_datetime(main_deadline) if not isinstance(main_deadline, datetime) else main_deadline

            task = {
                # 'uld_bags': [['dummy']] if pd.isna(row['uld_bags']) or row['uld_bags'] == "" else [[str(bag).strip()] for bag in row['uld_bags'].split(',')] if isinstance(row['uld_bags'], str) else [['dummy']],  # Convert to list of lists
                'uld_bags': str(row['uld_bags']).strip(),
                'Flight_no.': str(row['Flight_no.']).strip(),  # Ensure string and remove whitespace
                'AIBT': str(row['AIBT']).strip(),  # Ensure string and remove whitespace
                'inbound_flight_no': "" if pd.isna(row['inbound_flight_no']) else str(row['inbound_flight_no']).strip(),  # Empty string for NaN
                'outbound_flight_no': "" if pd.isna(row['outbound_flight_no']) else str(row['outbound_flight_no']).strip(), # Empty string for NaN
                'arr_dep': str(row['arr_dep']).strip(),  # Ensure string and remove whitespace
                'terminal': str(row['terminal']).strip(),  # Ensure string and remove whitespace
                'task_duration': row['task_duration'],
                'pickup_location': row['pickup_location'],  # Ensure string and remove whitespace
                'dropoff_location': str(row['dropoff_location']),  # Ensure string and remove whitespace
                # 'departure_location': row['departure_location'],  # Ensure string and remove whitespace
                'departure_location': "" if pd.isna(row['departure_location']) else str(row['departure_location']).strip(),
                'no_of_bags': row['no_of_bags'],
                'wide_narrow': str(row['wide_narrow']).strip(),  # Ensure string and remove whitespace
                'available_pickup_time': str(row['available_pickup_time']),
                'task_deadline': task_deadline,
                'main_deadline': main_deadline,
                'priority': row['priority'],
                'trip_order': row['trip_order'],
                'excel_index': str(row['excel_index']).strip()  # Ensure string and remove whitespace
            }
            tasks_data.append(task)

        except (ValueError, TypeError) as e:  # Catch potential errors during data conversion
            print(f"Error processing row: {row}")
            print(f"Error details: {e}")
            continue  # Skip to the next row

     # Convert the drivers data to the required format
    drivers_data = {}
    for _, row in drivers_df.iterrows():
        driver_id = str(row['index'])
        drivers_data[driver_id] = {
            'shift_starttime': row['shift_starttime'] if isinstance(row['shift_starttime'], str) else row['shift_starttime'].strftime('%Y-%m-%d %H:%M:%S'),
            'shift_endtime': row['shift_endtime'] if isinstance(row['shift_endtime'], str) else row['shift_endtime'].strftime('%Y-%m-%d %H:%M:%S'),
            # 'position': row['position']
            'position': "" if pd.isna(row['position']) else str(row['position']).strip()
        }
    return tasks_data, drivers_data # Return both tasks and drivers data



In [4]:
tasks_data
# convert tasks_data to dataframe 
tasks_df = pd.DataFrame(tasks_data)
tasks_df.info()


NameError: name 'tasks_data' is not defined

In [5]:
if __name__ == "__main__":
#     Exracting and transforming the data
    file_path = "2025-01-15_14_20_00_to_2025-01-15_15_20_00.xlsx"
    tasks_data, drivers_data = load_data(file_path)
    # print(json.dumps(tasks_data, indent=4))
    # print(json.dumps(drivers_data, indent=4))
    
#    Calling the Model
     # result = test_model(json_data,reassignment,num_driver_range=range(2,9), start_time = start_time,end_time = end_time, 
                   # lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)
    lock_chromo = {}
    reassignment= False  
    adjustable_lock_tasks = []
    start_time = '2025-01-15T14:20:00'
    
    
    
    
    
    
    
    
    
    
    
    end_time = '2025-01-15T15:20:00'

    result = model_call.test_model(tasks_data,reassignment,drivers_data,num_driver_range=range(2,9), start_time = start_time,end_time = end_time, 
                   lock_chromo=lock_chromo,adjustable_lock_tasks=adjustable_lock_tasks)

pick data


INFO:mvp_app.modules.task_assignment.rule_initialization:Total Drivers we have: 23
INFO:mvp_app.modules.task_assignment.rule_initialization:Current Drivers Dict: 
        shift_starttime        shift_endtime position
30  2025-01-15T10:10:00  2025-01-15T15:30:00         
39  2025-01-15T10:10:00  2025-01-16T00:00:00         
INFO:mvp_app.modules.task_assignment.rule_initialization:Starting with num_driver :2
INFO:mvp_app.modules.task_assignment.rule_initialization:Adjustable lock tasks: []


Adjustable lock tasks: []


INFO:mvp_app.modules.task_assignment.rule_initialization:ANT-CT: pickup time using crossTerminalBuffer for this task 87710
INFO:mvp_app.modules.task_assignment.rule_initialization:Task 87710 cannot be assigned to driver 30 as it exceeds the main deadline.
INFO:mvp_app.modules.task_assignment.rule_initialization:ANT-CT: pickup time using crossTerminalBuffer for this task 87710
INFO:mvp_app.modules.task_assignment.rule_initialization:Task 87710 cannot be assigned to driver 39 as it exceeds the main deadline.
INFO:mvp_app.modules.task_assignment.rule_initialization:Task ID 87710 is bypassing terminal check
INFO:mvp_app.modules.task_assignment.rule_initialization:Task 87710 is in a different terminal w.r.t driver 30--False
INFO:mvp_app.modules.task_assignment.rule_initialization:new_task_pickup_time for cross terminal task 87710 : 2025-01-15 14:30:00
INFO:mvp_app.modules.task_assignment.rule_initialization:Task ID 87710 is bypassing terminal check
INFO:mvp_app.modules.task_assignment.rule_

87710 cannot be assigned to any driver.
Incomplete chromosome: {'30': [], '39': []}
Cannot make a basic good schedule, please increase the number of drivers. Current number of drivers: 2
Adjustable lock tasks: []
87710 cannot be assigned to any driver.
Incomplete chromosome: {'30': [], '39': [], '34': []}
Cannot make a basic good schedule, please increase the number of drivers. Current number of drivers: 3
Adjustable lock tasks: []


...

pickup_location D41
dropoff_location 41
pickup_location D41
dropoff_location 41
pickup_location D41
dropoff_location 41
pickup_location D41
dropoff_location 31


INFO:mvp_app.modules.task_assignment.rule_initialization:Bypass Terminal Check is enabled for this task 87735


pickup_location C24
dropoff_location 18
pickup_location C24
dropoff_location 41
pickup_location C24
dropoff_location 41
pickup_location C24
dropoff_location 31
pickup_location D48
dropoff_location 18
pickup_location D48
dropoff_location 12
pickup_location D48
dropoff_location 41
pickup_location D48
dropoff_location 31
pickup_location C24
dropoff_location 18
pickup_location C24
dropoff_location 12
pickup_location C24
dropoff_location 19
pickup_location C24
dropoff_location 31
pickup_location D41
dropoff_location 18
pickup_location D41
dropoff_location 12
pickup_location D41
dropoff_location 19
pickup_location D41
dropoff_location 31
pickup_location D48
dropoff_location 18
pickup_location D48
dropoff_location 12
pickup_location D48
dropoff_location 19
pickup_location D48
dropoff_location 31


INFO:mvp_app.modules.task_assignment.rule_initialization:Best Result:


6 353.0
Data would be written to 'Model/model_result/2025_01_15_14_20_00_to_2025_01_15_15_20_00/on_time_20250204_083335.json'.


# test case json input creation
* preprocessing the input data

In [12]:
#  Take data for tasks
import pandas as pd
import json
import plotly.express as px
import copy
import datetime
import os
from datetime import datetime, timedelta
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy import create_engine
from mvp_app.models import Base

from dotenv import load_dotenv

load_dotenv()

DB_NAME="SATS-STG"
DB_HOST="satsstgdbcluster-instance-1.c1cv1hpzppoz.ap-southeast-1.rds.amazonaws.com"
DB_USER="postgres"
DB_PASS="satauroradbstg"

DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = create_engine(DATABASE_URL)

Base.metadata.create_all(engine)
conn = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()
import os

# Set an environment variable
os.environ['DB_NAME'] = DB_NAME
os.environ['DB_HOST'] = DB_HOST
os.environ['DB_USER'] = DB_USER
os.environ['DB_PASS'] = DB_PASS
# os.environ['ROOT_PATH'] = "/home/ec2-user/SageMaker/G-Ops_MVP"
os.environ['ROOT_PATH'] = "G:/G-Ops_MVP"

In [13]:
start_time = '2025-01-15T14:20:00'
end_time = '2025-01-15T15:20:00'
from sqlalchemy.sql import text

def fetch_task_from_to(start_time, end_time):
    query = text("SELECT * FROM tasks WHERE \
                 available_pickup_time >= :start_time \
                 AND available_pickup_time <= :end_time")
    result = conn.execute(query, {'start_time': start_time, 'end_time': end_time})
    columns = result.keys()
    result_dict = [dict(zip(columns, row)) for row in result]
    return result_dict

db_res = fetch_task_from_to(start_time, end_time)
print(len(db_res))
#  convert db_res to dataframe:
df = pd.DataFrame(db_res)
df.head()


31


,task_id,flight_id,pickup_location,dropoff_location,available_pickup_time,task_deadline,trip_time_id,create_date,write_date,adhoctask_id,priority,trip_order,task_status,main_deadline,create_by,write_by
0,87707,27427.0,D44,20,2025-01-15 14:24:00,2025-01-15 14:28:00,306.0,2025-01-15 06:39:38.241584,2025-01-15 06:39:38.241585,NaN,NaN,2.0,True,2025-01-15 14:47:00,System,AgentPlanner-TaskPriority
1,87710,27528.0,E26,31,2025-01-15 14:21:00,2025-01-15 14:32:00,589.0,2025-01-15 06:45:44.459013,2025-01-15 06:45:44.459013,NaN,NaN,1.0,True,2025-01-15 14:33:00,System,AgentPlanner-TaskPriority
2,87709,27528.0,E26,31,2025-01-15 14:36:00,2025-01-15 14:41:00,589.0,2025-01-15 06:45:44.459121,2025-01-15 06:45:44.459121,NaN,NaN,5.0,True,2025-01-15 14:48:00,System,AgentPlanner-TaskPriority
3,87728,27429.0,C24,12,2025-01-15 15:11:00,2025-01-15 15:27:00,184.0,2025-01-15 06:53:22.801563,2025-01-15 06:54:31.302063,NaN,18.0,3.0,False,2025-01-15 15:31:00,System,DelayTaskHandling-KFD
4,87736,27534.0,D41,18,2025-01-15 15:18:00,2025-01-15 15:22:00,357.0,2025-01-15 07:31:06.273916,2025-01-15 15:41:06.000000,NaN,1.0,2.0,True,2025-01-15 15:39:00,System,AgentPlanner-TaskPriority


In [14]:
def fetch_flight_from_to(db_res):
    flight_ids = [db["flight_id"] for db in db_res]
    
    if not flight_ids:
        return {}
    
    query = text("SELECT flight_id, flight_number, on_block, terminal, sta FROM flights WHERE flight_id IN :flight_ids")
    result = conn.execute(query, {'flight_ids': tuple(flight_ids)})
    
    # Get column names
    columns = result.keys()
    
    # Combine column names with row data to create dictionaries
    result_dict = {row[0]: {'flight_number': row[1], 'AIBT': row[2], 'terminal':row[3], 'sta': row[4]} for row in result}
    return result_dict
    
db_res_with_flight = fetch_flight_from_to(db_res)
for i in list(db_res_with_flight.items())[:2]:
    print(i)
len(db_res_with_flight)
db_res_with_flights_df =pd.DataFrame(db_res_with_flight).T
db_res_with_flights_df

res = []
for i in db_res:
    if i['pickup_location'] != 'ADHOCB':
        res.append(i)

db_res = res
#  conver db_res to dataframe:  
df = pd.DataFrame(db_res)
df.tail()

(27427, {'flight_number': 'TR659', 'AIBT': datetime.datetime(2025, 1, 15, 14, 10), 'terminal': '1', 'sta': datetime.datetime(2025, 1, 15, 13, 45)})
(27429, {'flight_number': 'TR101', 'AIBT': datetime.datetime(2025, 1, 15, 14, 54), 'terminal': '1', 'sta': datetime.datetime(2025, 1, 15, 15, 0)})


,task_id,flight_id,pickup_location,dropoff_location,available_pickup_time,task_deadline,trip_time_id,create_date,write_date,adhoctask_id,priority,trip_order,task_status,main_deadline,create_by,write_by
22,87712,27528,E26,31,2025-01-15 14:41:00,2025-01-15 14:53:00,589,2025-01-15 06:45:44.459095,2025-01-15 15:02:45.000000,None,2.0,4,True,2025-01-15 14:53:00,System,AgentPlanner-TaskPriority
23,87716,27570,E28,33,2025-01-15 14:35:00,2025-01-15 14:48:00,631,2025-01-15 06:45:44.458986,2025-01-15 06:45:44.458986,None,NaN,3,True,2025-01-15 14:53:00,System,AgentPlanner-TaskPriority
24,87721,27491,B2,47,2025-01-15 14:58:00,2025-01-15 15:08:00,459,2025-01-15 07:00:42.474962,2025-01-15 15:05:28.000000,None,2.0,5,True,2025-01-15 15:08:00,System,AgentPlanner-TaskPriority
25,87732,27430,D48,19,2025-01-15 15:10:00,2025-01-15 15:21:00,134,2025-01-15 06:43:29.031113,2025-01-15 15:21:46.000000,None,3.0,1,True,2025-01-15 15:22:00,System,AgentPlanner-TaskPriority
26,87727,27429,C24,12,2025-01-15 15:08:00,2025-01-15 15:12:00,184,2025-01-15 06:53:22.801515,2025-01-15 06:54:31.302063,None,12.0,2,False,2025-01-15 15:31:00,System,DelayTaskHandling-KFD


In [15]:
def model_input_transformation(task, flights):
        return {
                'uld_bags': '[["dummy"]]',  # Assuming this field is not in the DB, so setting it as an empty list
                'Flight_no.': flights[task['flight_id']]["flight_number"] if task['flight_id'] in flights else '',
                'AIBT': flights[task['flight_id']]["AIBT"].isoformat() if task['flight_id'] in flights else '',
                'inbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'outbound_flight_no': '',  # Assuming this field is not in the DB, so setting it as empty
                'arr_dep': 'arr',  # Assuming this field is not in the DB, so setting it as arr
                'terminal': flights[task['flight_id']]["terminal"] if task['flight_id'] in flights else '',  # Assuming this field is not in the DB, so setting it as empty
                'task_duration': 1,  # Assuming this field is not in the DB, so setting it as 0
                'pickup_location': 'ADHOCB' if task['pickup_location'] == 'T0' else task['pickup_location'] if task['pickup_location'] else 'ADHOCB',
                'dropoff_location': 'ADHOCB' if task['dropoff_location'] == 'T0' else task['dropoff_location'] if task['dropoff_location'] else 'ADHOCB',
                'departure_location': '',  # Assuming this field is not in the DB, so setting it as empty
                'no_of_bags': 1,  # Assuming this field is not in the DB, so setting it as 0
                'wide_narrow': 'WB',  # Assuming this field is not in the DB, so setting it as empty
                'available_pickup_time': task['available_pickup_time'].isoformat(),
                'task_deadline': task['task_deadline'],
                'main_deadline':task['main_deadline'] if task['main_deadline'] is not None else None,
                # 'priority': "", #Removed the field for now
                'priority': int(task['priority']) if task['priority'] is not None else 1,
                'trip_order': int(task['trip_order']) if task['trip_order'] is not None else None, 
                'excel_index': str(task['task_id'])  # Using task_id as excelIndex
            }
# Filter db_res by checking if Flight_id. is not None (not including ad_hoc tasks)
# db_res = [task for task in db_res if task['flight_id'] is not None]
# Apply the mapping function to each task
data = [model_input_transformation(task, db_res_with_flight) for task in db_res]
json_data = data
if len(json_data) != 0:
    print(db_res[0])
    print(f"Number of tasks = {len(json_data)}")
    print(f"Number of flights = {len({d['Flight_no.'] for d in json_data})}")

def sorted_priority(json_data):
    df = pd.DataFrame(json_data)
    df['available_pickup_time2'] = pd.to_datetime(df['available_pickup_time'])
    df_sorted = df.sort_values(by='available_pickup_time')
    df_sorted = df_sorted.reset_index(drop=True)
    df_sorted['priority'] = df_sorted.index + 1
    task_list = []
    for index, row in df_sorted.iterrows():
        task = {'uld_bags': row['uld_bags'] if row['uld_bags'] is not None else '', 
          'Flight_no.': row['Flight_no.'] if row['Flight_no.'] is not None else '',
          'AIBT': row['AIBT'] if row['AIBT'] is not None else '',
          'inbound_flight_no':row['inbound_flight_no'] if row['inbound_flight_no'] is not None else '',
          'outbound_flight_no':row['outbound_flight_no'] if row['outbound_flight_no'] is not None else '',
          'arr_dep': row['arr_dep'] if row['arr_dep'] is not None else '',
          'terminal':row['terminal'] if row['terminal'] is not None else '',
          'task_duration': row['task_duration'] if row['task_duration'] is not None else '',
          'pickup_location': row['pickup_location'] if row['pickup_location'] is not None else '',
        'dropoff_location': row['dropoff_location'] if row['dropoff_location'] is not None else '',
        'departure_location':row['departure_location'] if row['departure_location'] is not None else '',
        'no_of_bags': row['no_of_bags'] if row['no_of_bags'] is not None else '',
        'wide_narrow': row['wide_narrow'] if row['wide_narrow'] is not None else '',
        'available_pickup_time': row['available_pickup_time'],
        'task_deadline': row['task_deadline'].to_pydatetime(),
        'main_deadline': row['main_deadline'].to_pydatetime() if row['main_deadline'] is not None else '',
        'priority': row['priority'],
        'trip_order': row['trip_order'],
        'excel_index': row['excel_index']}
        task_list.append(task)
    return task_list

json_data = sorted_priority(json_data)
json_data_df =pd.DataFrame(json_data)
json_data_df.head()

{'task_id': 87707, 'flight_id': 27427, 'pickup_location': 'D44', 'dropoff_location': '20', 'available_pickup_time': datetime.datetime(2025, 1, 15, 14, 24), 'task_deadline': datetime.datetime(2025, 1, 15, 14, 28), 'trip_time_id': 306, 'create_date': datetime.datetime(2025, 1, 15, 6, 39, 38, 241584), 'write_date': datetime.datetime(2025, 1, 15, 6, 39, 38, 241585), 'adhoctask_id': None, 'priority': None, 'trip_order': 2, 'task_status': True, 'main_deadline': datetime.datetime(2025, 1, 15, 14, 47), 'create_by': 'System', 'write_by': 'AgentPlanner-TaskPriority'}
Number of tasks = 27
Number of flights = 9


,uld_bags,Flight_no.,AIBT,inbound_flight_no,outbound_flight_no,arr_dep,terminal,task_duration,pickup_location,dropoff_location,departure_location,no_of_bags,wide_narrow,available_pickup_time,task_deadline,main_deadline,priority,trip_order,excel_index
0,"[[""dummy""]]",SQ607,2025-01-15T14:16:00,,,arr,2,1,E26,31,,1,WB,2025-01-15T14:21:00,2025-01-15 14:32:00,2025-01-15 14:33:00,1,1,87710
1,"[[""dummy""]]",SQ957,2025-01-15T14:16:00,,,arr,2,1,E28,33,,1,WB,2025-01-15T14:21:00,2025-01-15 14:32:00,2025-01-15 14:33:00,2,1,87714
2,"[[""dummy""]]",TR659,2025-01-15T14:10:00,,,arr,1,1,D44,20,,1,WB,2025-01-15T14:24:00,2025-01-15 14:28:00,2025-01-15 14:47:00,3,2,87707
3,"[[""dummy""]]",TR659,2025-01-15T14:10:00,,,arr,1,1,D44,20,,1,WB,2025-01-15T14:26:00,2025-01-15 14:42:00,2025-01-15 14:47:00,4,3,87708
4,"[[""dummy""]]",SQ607,2025-01-15T14:16:00,,,arr,2,1,E26,31,,1,WB,2025-01-15T14:32:00,2025-01-15 14:38:00,2025-01-15 14:53:00,5,2,87711


## get drivers


In [ ]:
# Convert strings to datetime and adjust for 10 hours
# import datetime as datetime
from sqlalchemy import MetaData, Table, and_, cast, Date
from datetime import datetime, timedelta
start_time_n = datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%S') - timedelta(hours=24)
end_time_n = datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%S') + timedelta(hours=24)
# start_time = start_time - timedelta(hours=9)
# end_time = end_time + timedelta(hours=9)
print(f'shift_start_time={start_time_n}, end_time={end_time_n}')
def current_generate_drivers_from_db(start_time_n, end_time_n, session=session):
    # Define driver table structure
    print(f"Fetching drivers from the database for {start_time_n} to {end_time_n}")
    metadata = MetaData()
    drivers_table = Table('drivers', metadata, autoload_with=session.bind)
    driver_shifts_table = Table('driver_shifts', metadata, autoload_with=session.bind)
    print(f"Driver table columns: {drivers_table.columns.keys()}"
          f"\nDriver shifts table columns: {driver_shifts_table.columns.keys()}")

    try:
        filtered_driver_shift = session.query(driver_shifts_table).filter(
            and_(
                driver_shifts_table.c.shift_starttime >= start_time_n,
                driver_shifts_table.c.shift_endtime <= end_time_n
            )
        ).all()
        print(f"Number of drivers fetched: {len(filtered_driver_shift)}")

        # Get all_drivers in the format of: {'driver_id': {shift_start_time: '', shift_end_time: ''}}
        all_drivers = {str(driver.driver_id): {'shift_starttime': str(driver.shift_starttime), 'shift_endtime': str(driver.shift_endtime), 'position': None} for driver in filtered_driver_shift}

        #sort all_drivers based on shift_starttime:
        all_drivers = dict(sorted(all_drivers.items(), key=lambda x: x[1]['shift_starttime']))
        return all_drivers
    except Exception as e:
        session.rollback()
        print(f"Error fetching drivers: {e}")
        return {}

print(f"Querying drivers from {start_time_n} to {end_time_n} for tasks {start_time}, end_time{end_time}")
all_drivers = current_generate_drivers_from_db(start_time_n,end_time_n)
#  show all_drivers as dataframe:
all_drivers_df = pd.DataFrame(all_drivers).T

# keep only those drivers whose shift_end_time is greater than start_time:
# from datetime import datetime
start_time_n=start_time
# assuming start_time is a string in ISO format, convert it to datetime
start_time_n = datetime.fromisoformat(start_time_n)

all_drivers = {k:v for k,v in all_drivers.items() if datetime.fromisoformat(v['shift_endtime']) > start_time_n}
all_drivers_df = pd.DataFrame(all_drivers).T

len(all_drivers_df)
all_drivers_df

shift_start_time=2025-01-14 14:20:00, end_time=2025-01-16 15:20:00
Querying drivers from 2025-01-14 14:20:00 to 2025-01-16 15:20:00 for tasks 2025-01-15T14:20:00, end_time2025-01-15T15:20:00
Fetching drivers from the database for 2025-01-14 14:20:00 to 2025-01-16 15:20:00
Driver table columns: ['driver_id', 'create_date', 'write_date', 'staff_name', 'staff_id', 'active']
Driver shifts table columns: ['driver_shifts_id', 'driver_id', 'shift_starttime', 'shift_endtime', 'total_hours', 'break_starttime', 'break_endtime', 'actual_shift_starttime', 'actual_shift_endtime', 'schedual_leave', 'actual_leave', 'create_date', 'write_date', 'shift_date', 'ct_prep']
Number of drivers fetched: 41


23

In [31]:
num_driver_range = range(2, 9)
# Convert range object to a dictionary with min and max values
num_driver_range_dict = {
    "min": num_driver_range[0],
    "max": num_driver_range[-1]
}
lock_chromo = {}
adjustable_lock_tasks = []
reassignment = False

In [32]:
type(num_driver_range_dict)

dict

In [36]:
#  prepare Json file for the model having input 
# test_model(data,reassignment, num_driver_range,start_time=None, end_time=None,lock_chromo={},adjustable_lock_tasks=[]):

# Create the configuration dictionary
import json
from datetime import datetime

# Function to convert datetime objects in a dictionary
def convert_datetime_dict(item):
    if isinstance(item, dict):
        return {key: convert_datetime(value) for key, value in item.items()}
    return convert_datetime(item)

# Function to convert datetime objects to ISO format strings
def convert_datetime(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [convert_datetime(item) for item in item]
    return obj

# Convert the data list with datetime objects
converted_data = [convert_datetime_dict(item) for item in data]

# Create config with converted data
config = {
    "data": converted_data,
    "reassignment": reassignment,
    "num_driver_range": num_driver_range_dict,
    "start_time": start_time,  # Already a string based on the type output
    "end_time": end_time,      # Already a string based on the type output
    "lock_chromo": lock_chromo,
    "adjustable_lock_tasks": adjustable_lock_tasks
}

def save_config_to_json(config, filename="testcase1.json"):
    try:
        with open(filename, 'w') as f:
            json.dump(config, f, indent=4)
        print(f"Configuration successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving configuration: {str(e)}")

# Call the function to save
save_config_to_json(config)

def load_config_from_json(filename="testcase1.json"):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading configuration: {str(e)}")
        return None

Configuration successfully saved to testcase1.json


In [38]:
load_config_from_json("testcase1.json")

{'data': [{'uld_bags': '[["dummy"]]',
   'Flight_no.': 'TR659',
   'AIBT': '2025-01-15T14:10:00',
   'inbound_flight_no': '',
   'outbound_flight_no': '',
   'arr_dep': 'arr',
   'terminal': '1',
   'task_duration': 1,
   'pickup_location': 'D44',
   'dropoff_location': '20',
   'departure_location': '',
   'no_of_bags': 1,
   'wide_narrow': 'WB',
   'available_pickup_time': '2025-01-15T14:24:00',
   'task_deadline': '2025-01-15T14:28:00',
   'main_deadline': '2025-01-15T14:47:00',
   'priority': 1,
   'trip_order': 2,
   'excel_index': '87707'},
  {'uld_bags': '[["dummy"]]',
   'Flight_no.': 'SQ607',
   'AIBT': '2025-01-15T14:16:00',
   'inbound_flight_no': '',
   'outbound_flight_no': '',
   'arr_dep': 'arr',
   'terminal': '2',
   'task_duration': 1,
   'pickup_location': 'E26',
   'dropoff_location': '31',
   'departure_location': '',
   'no_of_bags': 1,
   'wide_narrow': 'WB',
   'available_pickup_time': '2025-01-15T14:21:00',
   'task_deadline': '2025-01-15T14:32:00',
   'main_de